<font size="6">Building Fast Queries on a CSV</font>

We will imagine that we are researchers monitoring the discourse about the climate changes on reddit, to find out what people think and discuss on this topic.

The goal of this project is to create a class that represent the comments about the climate changes on reddit. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from csv import reader

opened_file = open('/content/drive/MyDrive/Projeto Guiado 2.1/the-reddit-climate-change-dataset-comments.csv', encoding='UTF-8')
read_file = reader(opened_file)
rows = list(read_file)
header = rows[0]
rows = rows[1:]
print(header)
for i in range(2):
    print(rows[i])

['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw', 'created_utc', 'permalink', 'body', 'sentiment', 'score']
['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', '0.5719', '2']
['comment', 'imldbeh', '2qn7b', 'ohio', 'false', '1661990340', 'https://old.reddit.com/r/Ohio/comments/x2awnp/state_government_may_soon_kill_a_solar_project_in/imldbeh/', "Any comparison of efficiency between solar and fossil fuels is nonsensical at best and intentionally misleading at worst. In no universe is light -&gt; photovoltaic cell -&gt; electricity less efficient than

<font size="5">Reddit Class</font>

Let's start by implementing the constructor. It will take the name of the CSV file as argument and then read the rows contained in self.header and self.rows.

In [ ]:
from csv import reader
class Reddit():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename, encoding='UTF-8')
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
            
reddit = Reddit('/content/drive/MyDrive/Projeto Guiado 2.1/the-reddit-climate-change-dataset-comments.csv')
print(reddit.header)
print(len(reddit.rows))

['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw', 'created_utc', 'permalink', 'body', 'sentiment', 'score']
4600698


<font size="5">Finding a Comment Form the Id</font>

Throughout this project, we will make several improvements to the Comment class.<br>
The first thing that we will implement is a way to look up a comment from a given identifier. In this way, we can register and analyze the comment that we think is relevant.

The idea is proceprocess the data into a dictionary where the keys are the IDs and the values the rows.

In [ ]:
from csv import reader
class Reddit():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename, encoding='UTF-8')
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[1]] = row
            
    def get_comment_from_id(self, comment_id):
        if comment_id in self.id_to_row:
            return self.id_to_row[comment_id]
        return None

<font size="5">Analized Sentiment</font>

On our database, we have the analyzed sentiment of each comment, with the upper limit being defined as 1 and the lower limit as -1. This sentiments can be used to see how the comments were written, in a negative, neutral or positive way, which can inform us how this person sees the climate changes.

We will write a function that, given an interval between -1 and 1, return the messagens. The idea is proceprocess the data into a dictionary where the keys are the Sentiments and the values the rows.

In [ ]:
from csv import reader
class Reddit():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename, encoding='UTF-8')
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:1000]
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[1]] = row
            
    def get_comment_from_id(self, comment_id):
        if comment_id in self.id_to_row:
            return self.id_to_row[comment_id]
        return None
    
    def analized_sentiment(self, lim_inf, lim_sur):
        list_sentiment = []
        if lim_inf < -1 or lim_sur > 1:
            return None
        for rows in self.rows:
          if rows[-2] == '':
              rows[-2] = '0.0'
          sent = float(rows[-2])
          mess = rows[-3]
          if sent > lim_inf and sent < lim_sur:
              list_sentiment.append(mess)
        return list_sentiment

<font size="5">Comment Score</font>

With the score atribute, we can determine which comments are the most highly rated and, consequently, the most important one.

We will write a function that, given an score, return two messagens whose sum of the "score return" column value is equal to the score.

In [ ]:
from csv import reader
class Reddit():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename, encoding='UTF-8')
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:1000]
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[1]] = row
            
    def get_comment_from_id(self, comment_id):
        if comment_id in self.id_to_row:
            return self.id_to_row[comment_id]
        return None
    
    def analized_sentiment(self, lim_inf, lim_sur):
        list_sentiment = []
        if lim_inf < -1 or lim_sur > 1:
            return None
        for rows in self.rows:
          if rows[-2] == '':
              rows[-2] = '0.0'
          sent = float(rows[-2])
          mess = rows[-3]
          if sent > lim_inf and sent < lim_sur:
              list_sentiment.append(mess)
        return list_sentiment
    
    def score(self, score):
        score_messagens = []
        #for row in self.rows:
        #    if row[-1] == score:
        #        score_messagens.append(row[-1])
        for row1 in self.rows:
            for row2 in self.rows:
                if float(row1[-1]) + float(row2[-1]) == score:
                  score_messagens.append(row1[-3])
                  score_messagens.append(row2[-3])
                  return score_messagens
        return -1

In [2]:
!pip install pytest pytest-sugar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
%%file test_data.py
import pytest

from csv import reader
class Reddit():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename, encoding='UTF-8')
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:1000]
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[1]] = row
            
    def get_comment_from_id(self, comment_id):
        if comment_id in self.id_to_row:
            return self.id_to_row[comment_id]
        return None
    
    def analized_sentiment(self, lim_inf, lim_sur):
        list_sentiment = []
        if lim_inf < -1 or lim_sur > 1:
            return None
        for rows in self.rows:
          if rows[-2] == '':
              rows[-2] = '0.0'
          sent = float(rows[-2])
          mess = rows[-3]
          if sent > lim_inf and sent < lim_sur:
              list_sentiment.append(mess)
        return list_sentiment
    
    def score(self, score):
        score_messagens = []
        #for row in self.rows:
        #    if row[-1] == score:
        #        score_messagens.append(row[-1])
        for row1 in self.rows:
            for row2 in self.rows:
                if float(row1[-1]) + float(row2[-1]) == score:
                  score_messagens.append(row1[-3])
                  score_messagens.append(row2[-3])
                  return score_messagens
        return -1

reddit = Reddit('/content/drive/MyDrive/Projeto Guiado 2.1/the-reddit-climate-change-dataset-comments.csv')

#@pytest.fixture(scope="session") 
def test_1():
    """
    Test evaluation for id 'imlddn9'
    """
    reddit = Reddit('/content/drive/MyDrive/Projeto Guiado 2.1/the-reddit-climate-change-dataset-comments.csv')
    assert reddit.get_comment_from_id('imlddn9') == ['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', '0.5719', '2']

def test_2():
    """
    Test evaluation for id imldbeh
    """
    reddit = Reddit('/content/drive/MyDrive/Projeto Guiado 2.1/the-reddit-climate-change-dataset-comments.csv')
    assert reddit.get_comment_from_id('imldbeh') == ['comment', 'imldbeh', '2qn7b', 'ohio', 'false', '1661990340', 'https://old.reddit.com/r/Ohio/comments/x2awnp/state_government_may_soon_kill_a_solar_project_in/imldbeh/', "Any comparison of efficiency between solar and fossil fuels is nonsensical at best and intentionally misleading at worst. In no universe is light -&gt; photovoltaic cell -&gt; electricity less efficient than light -&gt; entire food chain -&gt; biomass -&gt; decomposition -&gt; millions of years of geothermal heat and pressure -&gt; extraction -&gt; refining and transport -&gt; burning -&gt; turbine generator -&gt; electricity.\n\nUgly? More ugly than power lines, roads, apartment buildings, tractors, fences, etc etc etc? You're simply not used to it. Also, who cares? Sure, millions of people died in the displacement from sea rise, famines cause by climate change, and wars from resource scarcity, but thank God we didn't have to look at those solar panels occasionally.\n\nOffshore fracking? Fucks sake. Have you already forgotten the deepwater horizon disaster? Think about the damage that did to the gulf coast, add a few million barrels of highly toxic fracking fluid to the mix, and there you have our next major environmental disaster courtesy of offshore fracking. But oh man, those solar panels sure are ugly.\n\nStop repeating the lies fed to you by the people getting rich from fossil fuels, who couldn't care less about you or your descendants as long as they get theirs. We can deal with the less environmentally friendly aspects of solar power. We cannot deal with the consequences of continuing to pump carbon into the atmosphere. Y'all need to grow the fuck up and stop whining about the view.", '-0.9877', '2']

def test_3():
    """
    Test evaluation for sentiment between -0.5 and 0.5
    """
    reddit = Reddit('/content/drive/MyDrive/Projeto Guiado 2.1/the-reddit-climate-change-dataset-comments.csv')
    assert reddit.analized_sentiment(-0.5,0.5) == ["I'm honestly waiting for climate change and the impacts of that to kick some fucking sense into people. But who am I kidding itll still just be more of the poor suffering while the rich claim victim hood for handouts while letting us all starve. Its honestly hard some days to not just give up, and I truly wonder if and when anything will ever actually be done.", "Not just Sacramento. It's actually happening all over the world. Climate change is real, believe it or not.", 'Naaa how could anyone be mad at a face like that... Must definitely be climate change', "They need to change laws so it's more worth selling agriculture products in the US rather than export it.  They also need to change laws so there are monetary penalties for growing crops that are not particularly viable to an area's natural climate.  As it stands right now, my neighbor makes double the price per head of cattle by exporting out of country than he would selling right here.  All the people complaining about climate change on here should probably be complaining about this too.", "Blaming environmentalists for the failure of nuclear energy world wide is just such a weird take, when did governments/companies ever give a fuck about their opinion? Never as seen by the small problem of climate change that we have now. But what do they care about? MONEY! \n\nNuclear energy was never cheap like the industry promised in the 70s. In the past it wasn't competitive against fossil fuels and now it isn't against renewables.", "&gt;Not climate change mind you\n\nHi. I have a degree in Climate Science. While California has been wetter in the past 150 years compared to the state's recent paleoclimatological history, the current aridification of the American west is driven by human-caused climate change and excessive water usage.", 'Climate change is not "staring" you in the face, it has been in your face from the day you were born and will stay there \'till you\'re 6 feet under. You won\'t be there to face "the end of times", neither will the children of your children. Fixing thd climate will take as long as we\'ve been destroying it.\n\nWe will never call this good times. Comparing economy with climate makes no sense.', 'The overturning of Roe V Wade, children are literally forced to keep a fetus in their body that cannot live after being raped because it still has a heart beat with a significant chance of permanently harming or killing the child mother (10 years old 10 YEARS OLD!!! in the last example I heard about). This is beyond disgusting, and if something like this can pass (and still find approval), then the sky is the limit. Members of the supreme court have openly said they are going after inter racial marriage, and same sex marriage. \n\nGerrymandering is so pervasive now its destroyed the ability of congress to actually represent its constituents. \n\nWe have members of the government openly spouting blatant hate speech and pushing easily debunked conspiracy theories. Yet the majority of their party still supports them. \n\nHigher education is absurdly expensive, in the 1960\'s you could go to a university for less than 1k a year, adjusted for inflation. Before that believe it or not university was actually free all over the place. \n\nCorporations continue to gain more and more power, the wealth gap in the US is like that before the French revolution. Its literally multiplied in the last couple decades. \n\nStaple foundational books are being taken out of school library\'s. Laws are going up saying that school library\'s have to show "both sides" ...the example of the holocaust was brought up, and yes there needs to be a pro-holocaust perspective. \n\nTo pass the "biggest green climate change helping" legislation in ages they lumped in TONS of regressive fossil fuel industry help. \n\nI don\'t remember the exact specifics (its been a long day), but basically the court is going to be possibly allowing those that normally transfer the election results to decide that the election was rigged and to cast the vote for someone that did not win the election. (I wish I was making this stuff up)\n\n...the list goes on and on, I\'m tired. Not saying, there have not been positive changes, however there are some huge shifts going on that honestly could bring the US to its knee\'s its not forgone, but it ABSOTLUTELY should not be ignored.', "You should check out Paul Nicklen's (the guy in the video) Instagram account. The man basically lives in Antarctica and runs a non profit called Sea Legacy that is working to stem the tide of climate change and ocean pollution", 'Damn that climate change', 'Correct, "climate change" is not the same thing as the air quality index in Shanghai.  No, I don\'t think you get it.', "I'm sure it's climate change.  Probably has nothing to do with the vaccine.", 'Climate Change', "Federal protection was introduced to protect them (and many other birds) from the feather and egg trades, so anything else that negatively effects them like habitat loss or loss of food (such as through development or climate change) isn't prohibited.\n\nedit:changed is to isn't. woops.", 'Sheesh, these folks would have you believing snow is black and climate change makes it white.\n\nNothing makes sense', "Doesn't change the fact that this is short sighted, because the effect of climate change are going to be many many many times costlier than any amount of nuclear plant we could build.", 'I can’t speak for your exact scenario but the SYGAV unit I have ties into the CANBUS system and I can change the climate control settings through the touchscreen AND via the climate control buttons below.', 'The first part is what they should say.  The second part makes no sense at all.  It would cost more to move everyone into space then it would to fix climate change.', 'Yeah I remember an article somewhere suggesting climate change was causing an increase in this and other heart-related issues in young people. Not sure if any formal study was done', 'I’m sure their first suggestion was a MASSIVE reduction in annual military spending an audit of existing waste and a pledge to not topple democratically elected leaders in the areas most likely to be destabilized by climate change right?', 'I think we should develop underwater cities, aside from protection from climate change extremes, there may be planets where we have to live under water or under ice, or under liquid methane etc.', 'Ambient particular matter pollution is not climate change. It’s pollution. The actual temperature change or weather extremities do not meaningfully contribute to the risk factor at all.\n\nTo equate particulate pollution to climate change is very disingenuous imo.', '&gt;This and this period of time doesn\'t seem to be any different.\n\nThe difference is that in the past machines replaced human physical labour and pushed humans to leverage their mental intelligence to justify their labour value instead. Now artificial intelligence replaces human mental intelligence. What do humans have to leverage next?\n\nThe main point is that artificial intelligence is a fundamentally unique type of automation.\n\nBut often people fall into the fallacy of believing that "human society always eventually advances in a positive direction" is an unfallable law of nature based on an insignificant 100k year record which biases them towards over-valuing historical trends and prevents them from correctly appraising new things.\n\nSimilar reason for why people have difficulty recognizing how human intervention can drive climate change and mass extinction of millions of species. It is difficult to recognize how advancement enables human actions to negatively impact the environment on a planetary scale. Especially so when historical biases drive people to assume that humans are, and have always been mostly insignificant when discussing planetary scale events.', "Oh we're absolutely fucked for climate change. \n\nEven if there were the political will to turn down industry money and do anything meaningfully positive there, every thing we attempt to do to improve things going forward will be called tyranny by the fucking mush brains who have propagated in the age of social media and disinformation", 'Ironically these are the same people who pretend to be so concerned with climate change/ etc.', 'And Climate Change.  \n\n\nNeed more people to take the bus, bike or walk.   Only way we will survive is if there are less cars and more bikes.', "they're not wrong... people try to debunk climate change and the fossil record with the bible. *And they're serious about it*. ***And they teach their kids about it.***", 'Greatest country on earth strikes again. Major cities without water? Whole states power grid buckling under self-imposed climate change? A million plus dead from a pandemic? Sorry, unless it can produce immediate, infinitely-growing profit for someone who doesn’t need money, the American plan is to just do nothing. We’re told that government serving the quality of life of the citizens would be socialism, which they tell us is an unforgivable evil. \n\nBut we need to funnel another 1.5 billion to our arms manufacturers, betcha that’ll have no difficulty immediately passing through congress.', "Forget this milquetoast center-left rainbow capitalism. If this won't stop climate change, wealth inequality, consumerism, wage theft, private prisons, excessive working hours, food shortages, expensive rent, expensive healthcare, or cities designed around cars, then I'm not in.", 'Racist! Semi-Fascist! Climate change! Reeeeeeeeeee', 'I think you need to deal with this asap and consider that the near future holds even more aggressive rain than we get now\n\n&amp;#x200B;\n\nyay climate change.', 'Is anybody else paying attention to the insane weather events happening in china? Is climate change the black swan that takes us all out?', 'Climate Change and too many electric cars', 'Europe, Pakistan, China, the western US, and more are all dealing with unprecedented accelerating climate change driven droughts, floods, and energy crises. I have a difficult time seeing why so much ink is spilled on AI-gone-wrong extinction scenarios given the massive ongoing Anthropocene extinction event.', 'What do you think the definition of climate change is?  lol', 'This has been a strong ecumenical view since forever, yet individual Christians keep doing stupid stuff like not believing in climate change.', 'The actual transformation necessary for us to stop climate change within our lifetimes', "Was alive for 58 minutes. The original [post](https://np.reddit.com/r/worldnews/comments/x2qgek/claims_of_torture_of_china_uyghurs_credible_un/).\n\nThe deleted submission has been flagged with the flair **Already Submitted**\n\nI tried to find other articles: \n\n| Post | Confidence | Old | Score |\n|:- |-: |:-: |:-: |\n| [Claims of torture of China Uyghurs are credible - UN](https://np.reddit.com/r/worldnews/comments/x2q3z5/claims_of_torture_of_china_uyghurs_are_credible_un/)^(hot) | 103.0% | an hour | [+639/c98] |\n| [Claims of torture of China Uyghurs are credible - UN](https://np.reddit.com/r/worldnews/comments/x2q76t/claims_of_torture_of_china_uyghurs_are_credible_un/)^(hot) | 103.0% | an hour | [+30/c4] |\n| [UN rights chief 'under tremendous pressure' over report on China's Uyghurs](https://np.reddit.com/r/worldnews/comments/wxcpop/un_rights_chief_under_tremendous_pressure_over/)^(removed) | 45.984% | 6 days | [#83/+23/c8] |\n| [Point of no return: crunch time as China tries to fend off property crash](https://np.reddit.com/r/worldnews/comments/wzwelq/point_of_no_return_crunch_time_as_china_tries_to/)^(removed) | 43.017% | 3 days | [#14/+258/c58] |\n| [Defiant Taiwan says it has 'resolve' to stand up to China on battle anniversary](https://np.reddit.com/r/worldnews/comments/wvkn35/defiant_taiwan_says_it_has_resolve_to_stand_up_to/) | 42.795% | 8 days | [+76/c1] |\n| [Its largest lake is so dry, China digs deep to water crops](https://np.reddit.com/r/worldnews/comments/wvymhs/its_largest_lake_is_so_dry_china_digs_deep_to/)^(removed) | 42.736% | 8 days | [#44/+48/c32] |\n| [Taiwan vows to 'counter-attack' if China's forces enter its territory](https://np.reddit.com/r/worldnews/comments/x26eks/taiwan_vows_to_counterattack_if_chinas_forces/)^(removed) | 42.607% | 16 hours | [#42/+31/c4] |\n| ['Man of the Hole': Last of his tribe dies in Brazil](https://np.reddit.com/r/worldnews/comments/x0qid1/man_of_the_hole_last_of_his_tribe_dies_in_brazil/) | 42.424% | 2 days | [+18/c3] |\n| ['Man of the Hole': Last of his tribe dies in Brazil](https://np.reddit.com/r/worldnews/comments/x0lsnf/man_of_the_hole_last_of_his_tribe_dies_in_brazil/) | 42.424% | 2 days | [+70/c8] |\n| ['Man of the Hole': Last of his tribe dies in Brazil](https://np.reddit.com/r/worldnews/comments/x0yvur/man_of_the_hole_last_of_his_tribe_dies_in_brazil/)^(removed) | 42.424% | 2 days | [#70/+31/c6] |\n| ['Man of the Hole': Last of his tribe dies in Brazil](https://np.reddit.com/r/worldnews/comments/x0r50i/man_of_the_hole_last_of_his_tribe_dies_in_brazil/) | 42.424% | 2 days | [+59/c4] |\n| [Solomon Islands Bans U.S. Navy Ships From Ports After China Security Deal](https://np.reddit.com/r/worldnews/comments/x1f8et/solomon_islands_bans_us_navy_ships_from_ports/) | 41.364% | a day | [+179/c35] |\n| [Zelenskiy vows to restore Ukrainian rule in Crimea](https://np.reddit.com/r/worldnews/comments/wvmk83/zelenskiy_vows_to_restore_ukrainian_rule_in_crimea/) | 40.816% | 8 days | [+7978/c482] |\n| [Tuvalu minister says both China, U.S. needed for climate change action](https://np.reddit.com/r/worldnews/comments/wy72hh/tuvalu_minister_says_both_china_us_needed_for/) | 40.593% | 5 days | [+4/c0] |\n\n\n---\n\n*^(This info could also be completely unrelated or unhelpful in which case I apologize. I'm still learning.)*\n\n^^Show ^^me ^^what ^^you ^^got ^^/u/coverageanalysisbot", "Hear that! I'd cry a river, but they're all dried up due to climate change", 'climate change?', "Here's some fun. Take a look at snowfalls and green Christmases in Canada. Can you spot a trend? Almost every city is experiencing less snow on average and more Green Christmases. It's especially worse in some areas.\n\n[https://www.canada.ca/en/environment-climate-change/services/weather-general-tools-resources/historical-christmas-snowfall-data.html](https://www.canada.ca/en/environment-climate-change/services/weather-general-tools-resources/historical-christmas-snowfall-data.html)\n\nCheck out change of frequency, snow depth in the past, and green Christmases.\n\nSarnia is nice, 6 in 30 years, then 21 green Christmases in the past 30.\n\nHow many cities on there are having more snow now?", 'Probably not too much. Sindh Province has received 784% higher than average rain while Baluchistan received 500%. In Monsoon season. So yeah, that’s gonna do damage no matter what. Climate change almost certainly caused that.', 'And I’m sure nothing you do contributes to climate change', 'I can\'t say how bad over the next year. Year by year you can have pretty drastic differences, but it\'s the long-term trend you have to look at. But 100 years from now, I don\'t think you\'re going to find many humans on this planet, if you find any... We\'re very rapidly approaching a point where a virologist with the motivation and modest means can produce a handful of deadly, contagious viruses and wipe out most of mankind. This isn\'t crazy talk. You can order DNA on the internet. You can buy the equipment to manufacture viruses for less than the cost of a house. We have complete genomes (publicly available info) for thousands of viruses.\n\nSome might ask, "why would someone do that?" I\'d say, "Ask Stephen Paddock, Jim Jones, David Koresh, and Jin Ruchao, among others." You can\'t tell me Stephen Paddock wouldn\'t have wiped out mankind given the opportunity. So it\'s not a matter of IF it will happen. It\'s simply a matter of when a virologist gets angry enough.\n\nAnd if by some chance that just doesn\'t happen (maybe civilization collapses first), then we\'re screwed in a number of other ways. Every day there are stories about how climate change is worse than they originally predicted. Today from Scientific American, "This Hot Summer Is One of the Coolest of the Rest of Our Lives" An article on CBS today about lakes in the Arctic disappearing, "Scientists long expected that Arctic lakes would expand with climate change as ground ice continued to melt and climate models showed that drying would not be seen until at least 2060 or 2150." Oops, guess they botched that prediction pretty badly. All the glaciers are disappearing way faster than they ever predicted. Nobody predicted the massive methane emissions from thawing permafrost, and thawing permafrost wasn\'t anything we were expecting to see in the 2000s, if you asked 20 years ago.\n\nA lot of people find this really depressing. I mean, I\'ve done a good bit of research in this and I don\'t really see a silver lining. Our best case scenario is that there will be a massive loss of life and maybe some people will survive long-term, but I think it\'s unlikely. I\'m way past the emotional side of this. Until someone can show me something super-promising, I see this as more or less, a fact waiting to happen.\n\nDid I mention the plummeting fertility rates for men? Between 1930 and 1992, sperm counts in men in America dropped about 50%. They continue to fall and those rates are now dropping worldwide. Within a few decades, most men will be effectively infertile. So you tag that on to all the other stuff going wrong and it\'s going to be hard to survive.', "The failure of Yucca Mountain, Fukushima, the leaks found at San Onofre, Chernobyl and the fact that nuclear sites can apparently be used to store your troops and ammo does not make me feel reassured.     \nTell me where you're going to store the waste, tell me how you're going to remove the fissile material under extreme circumstances, how to stop a meltdown, and how to protect it from being used as enemy strongholds and you've got my support.  Specially with sea level rise and extreme climate change your can't guarantee that the location you pick today to build a plant will be safe tomorrow.", "Unfortunately it's likely to get colder there. The last time the northern hemisphere warmed up it tossed Europe into an ice age.\n\nClimate change is weird like that.", 'Well, according to this [source](https://news.stanford.edu/2016/09/01/extreme-weather-winters-becoming-common/), this [source](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5928063/), this [source](https://www.science.org/doi/10.1126/science.abi9167), and this [source](https://journals.ametsoc.org/configurable/content/journals$002fclim$002f33$002f18$002fjcliD190994.xml?t:ac=journals%24002fclim%24002f33%24002f18%24002fjcliD190994.xml&amp;tab_body=fulltext-display), (these are all academic sources mind you) it’s happening everywhere. Global climate change will mean more extreme winters.\n\nThat frozen stuff also won’t necessarily be arable for crops, and you also have to factor in the effects of [soil degradation](https://link.springer.com/article/10.1007/s10661-020-08429-5), [water resources](https://www.researchgate.net/profile/J-Tarafdar/post/How_climate_change_impact_on_water_situation_in_India/attachment/5f4643f36a5a0300017b1b09/AS%3A928740653682689%401598440435831/download/JEEpaper-ImpactofClimateChangeonWaterResourcesinIndia.pdf), and [other issues](https://www.mdpi.com/2071-1050/13/3/1318/htm) that will affect our current amount of arable land thanks to climate change. Events such as flooding as this video shows end up stripping the soil of nutrients needed to grow crops, and these events are much more frequent under climate stress.', 'Climate change is at its core a naturally occurring phenomenon.', "So the one I have is the UK one. The fan speed control and temperature are all on the screen. You press the climate button and it takes you to a separate page on the screen. That's why I'm unsure if I can change it.", 'Because of projected future scarcity as climate change ravages the worlds ecosystems?', 'The irony here is- a majority of these people probably claim to be concerned about climate change.', 'The Intergovernmental Panel on Climate Change (IPCC) provide executive summaries of all their reports, with the idea of making the language simple and understandable. \n\nThey are completely objective, and the only people questioning their objectivity are those with vested interests themselves. \n\nhttps://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_SummaryForPolicymakers.pdf', 'We should not be flying on fossil fuel commercially, considering climate change.', 'Unfortunately you can’t exactly slay climate change.', '“1,200 scientists out of hundreds of thousands disagree with the verbiage of “emergency”, when speaking on climate change, but still very much believe in climate change and global warming” ….is what the headline should have said.\n\nI found and read the article. In a nutshell-  the scientists believe in climate change, but not the non-uniform measures for assessing what is or isn’t considered a climate “emergency”. And it will be important moving forward to distinguish different climate changes based on a trusted method.\n\nI fucking hate headlines and the way news is presented + consumed nowadays. He posted this with NO context 😂 and it doesn’t even defend the fact that global warming isn’t real sooo.. (I’m assuming that’s what his rectangular self was getting at)', 'https://www.bbc.com/future/article/20210816-how-water-shortages-are-brewing-wars\n\nhttps://www.bloomberg.com/news/articles/2021-11-19/global-warming-is-destroying-crops-can-it-be-stopped-in-time-kw65js74\n\nhttps://press.un.org/en/2018/gaef3499.doc.htm\n\nhttps://www.usnews.com/news/best-countries/articles/2021-10-29/how-climate-change-may-increase-global-conflicts\n\nhttps://www.pnas.org/doi/10.1073/pnas.0703073104\n\nhttps://www.theatlantic.com/science/archive/2022/03/nuclear-war-would-ravage-the-planets-climate/627005/\n\nhttps://www.theguardian.com/environment/2021/jan/13/top-scientists-warn-of-ghastly-future-of-mass-extinction-and-climate-disruption-aoe\n\nhttps://www.smithsonianmag.com/science-nature/without-action-on-climate-another-mass-extinction-event-will-likely-happen-in-the-worlds-oceans-180979991/\n\nhttps://www.bbc.com/news/science-environment-62378157', 'So here’s me thinking about so many 20 somethings complaining about climate change (FTR I’m 32 and yes believe in climate change) and here is a festival (more than likely) attended by mostly 20 somethings and look at the WASTE. The post pilfering will take care of maybe a 4th, and the rest will end up in a land fill .. just absolutely gross.', 'Man not even Washington state requires smog testing and they support climate change measures more then anyone. It’s just another tax…', "Let's say I own part of a solar panel company but am under the cap.\n\nI vote through climate change measures which increases the companies worth and I'm now above the cap. Do I have to resign immediately? What if the price later goes down?", "See that would've been a GOOD character arc but he would have to support fighting climate change though", "No, it's pretty simple really. You don't want there to be any actual action on climate change, you'd prefer to just post about it and feel progressive. For there to be any actual action you would need to do what needs to be done to actually make change and you aren't interested in that, 43% isn't good enough so you'll happily take Morrison and 25% reduction as that allows you to continue making pathetic attempts at using rape to feel superior.", 'Over population is probably the single biggest contributor to Climate change.', "And her state is not about to become uninhabitable from drought, because climate change isn't real!", 'Climate change started well over 40 years ago.', "Purely for midterm messaging. \n\nIt's ultimately a climate change and healthcare bill that will have only modest, long-term impact on inflation, mostly via the medicare price negotiation feature and diversification of the energy grid, but no real short term impact. \n\nThat said, I'm supportive of about 80% of the bill so I don't really care what they call it.", 'Climate change being our fault, that we need to change the whole system.', "How many more years of oil do you think are available in the quantities that we are accustomed to?\n\nOne silver lining of petroleum-induced climate change is that we can drill in the Arctic.  Even so, how much longer can we burn this finite resource until there is simply no more oil to be had?\n\nDo we care about the long-term viability of modern civilization?  If so, perhaps we should give serious thought to  this question and it's implications.", 'Would be nice if African and South American were doing more to fight climate change', 'Online misinformation about climate change is a super category I guess', 'There’s room for more than them. But Nixon also created the EPA and did a lot for climate change and the environment. Gotta take the good with the bad.', 'We deserve climate change.', 'mf literally romanticizing climate change', 'Climate change', 'Serious question for these troubling times: how is the best way to profit from redditors who think that there are like ten years before climate change kills us all', 'Corporations that invest millions of dollars into campaigns focused on making people feel individually responsible for trash, pollution, and other climate change contributors in an effort to avoid real change and legislation that would actually hold them rightfully accountable.', 'The US had an exodus of natural gas prior, its almost as if we could… start them back up and sell them to the EU. But ya know… “climate change”', 'Climate change. Our emissions. Our waste', 'If humans give up meat for climate change, we deserve extinction.', 'Some of these people are the same ones screaming about climate change.', "I mean, there's evidence the Earth is flat, too. Just look at the horizon. Does it look curved? \n\nThe evidence against man-made climate change is similar - it requires a myopic view that either ignores or distorts the vast bulk of the scientific evidence. \n\nScientists spent much of the 1900s hashing this issue out, and finally got good enough data in the 1950s-1970s to finally come to a conclusion. And its not like another 50 years of even *more* data has changed that. On the contrary, it's firmed up even more.", 'Although the film crew swore up and down they didn\'t do *anything* to upset the walruses? The facts are different:  \n(This is aside from the fact they spliced footage from TWO different areas and pretended it was one area)  \n- there were polar bears at the top.  \n- they buzzed the walruses with drones frequently.  \n- they set up cameras FAR too close to the ramp the walruses use to get down from there  \n- walruses have been using that area (up at the top) for as long as we know, it isn\'t a "new thing"  \n- walrus numbers shrink and grow quite a bit, and numbers have been duly noted by every explorer of the area. Sometimes huge \'haul outs\' like what was filmed, other times hardly any, long before climate change started.  \n- early explorers noted how polar bears liked to scare walruses over cliffs because in huge herds they\'re quite dangerous, so it\'s safe and easy :&gt;\n\nAttenborough refused to discuss the matter, ever. BUT he did use footage (taken at that time, not new stuff) in a later documentary showing the bears &amp; drones that the film crew swore weren\'t there...', 'When climate change shuts down the electricity grid due to flooding and unbearable heat… maybe we’ll have our jobs back.', 'Michael Crichton (the Jurassic Park guy) was a trained scientist. His last book was a footnoted novel "State of Fear" which covered climate change.  I remember it talking about the difference in temperature rise in the last 100 years in a city that has been cemented/ asphalted in verse a rural area 100 miles away which hadn\'t been cemented in, for example.', 'The best of times, the worst of times - Paul Behrens. It sketches an outlook for the coming period through a pessimistic and an optimistic lense. Good read on the various subjects within climate change', "You might want to look outside, because most people do not know this.  Most people think climate change is a thing that happens in like 100 years, and that we can stop it by then.  They're too blind to see it's happening now.", 'Coastal lowland tiles are ones that can flood with climate change. This thread has some pictures that show examples: https://forums.civfanatics.com/threads/climate-change-quirks.653220/', 'That’s when you tune them out, though they are more civilized. Their problem is they are the educators. Then, when you say education should be free or open to all classes (including the ones they themselves build, although they are no terrorists, big oil (big pharma, all the bigs), you will find resistance. The most powerful have the most money, the least debt and that is of importance over the others in their class. Either that, or it threatens their business and future, which is always ironic considering some would die for climate change yet never consider those who lose jobs from coal plants. \n\n\nHas anyone figured out who to listen to/ recommend anything once you are past the Conservatives, the Democrats, and the liberals? I know there are good people in all political groups, but it does get pretty ugly once you see through liberals. Ugly and lonely… I try to say that it’s my problem for expecting or searching for an ideology without contractions, but admitting that I actually do believe we shouldn’t have these barriers has lost me many friends over the years. \n\n\nSuggestions welcome. Please no trolling.', "J'ai mit le Congo (sous entendu RDC) par préjugé, comme j'aurai pu mettre presque n'importe quel pays d'Afrique sub-saharienne.\n\nSi on s'en tient à des sondages réalisés sur l'Afrique, il y a bien la moitié de la population qui ne sait pas ce qu'est le réchauffement climatique (c'est pas tant du déni mais plutôt qu'ils sont juste pas au courant). [https://theconversation.com/africas-first-continent-wide-survey-of-climate-change-literacy-finds-education-is-key-169426](https://theconversation.com/africas-first-continent-wide-survey-of-climate-change-literacy-finds-education-is-key-169426)\n\nAlors la RDC n'est pas le pire dans le groupe vu qu'ils ont 80% d'alphabétisme, mais si on prend le Niger avec ses 20% d'alphabétisation, c'est pas étonnant que peu de gens soient au courant. Tous les pays de l'étude de ce post ont un taux d'alphabétisation supérieur à 90%, ça fait une grosse différence.\n\nEdit : ce que je voulais dire c'est que si on doit se comparer à des pays qui ont 60% d'alphabétisation ou des dictatures pétrolières alors c'est déjà placer la barre beaucoup trop bas, dans tous les cas c'est la honte.", 'Turns out when you ignore environmentalists, scientists and evidence about climate change and refuse to invest in renewables on a National scale, when the oil starts to run out and the air is nasty to breathe, the only option you have left is nuclear or societal breakdown. \n\nAnd I’d still pick societal breakdown. \n\nEnvironmentalists are not choosing nuclear because its clean or safe. Lobbyists have made sure it’s the only option on the table.', "You have a real problem with reading comprehension and putting words in people's mouth. You haven't given me a shred of proof either. You linked me a graph about CO2 levels, which I never questioned.  I also never said man made climate change doesn't exist. In fact, I said the opposite. Try again Karen.", 'Global warming/climate change and pollution in general. This should not be a political issue.', 'If you ever hear about "1,200 scientists agree climate change is a hoax," most of those guys are engineers, too.', 'Could a permanent lake created by such floods bring humidity and change the climate in the region for a certain period of time?', "Any support for him is completely fabricated. The Democratic Party is worthless. They have nothing to offer. Their biggest claim is they hate Trump still, they impeached Trump and Black Lives matter, climate change. It's a dystopia.", 'Kinda hard to stabilize the home insurance market for a future underwater land mass.\n\nSometimes wealthy people pretend they\'re "skeptical" of climate change in public to get the rubes to keep voting red, but when it\'s time to really get down to it they know much of Florida\'s days above water are numbered.', 'Pakistan foreign minister:\n&gt;\t"And while we understand that the new reality of climate change means more extreme weather, or monsoons, more extreme heat waves like we saw earlier this year, the scale of the current flood is of apocalyptic proportions. We certainly hope it\'s not a new climate reality."', "You are correct. I have no concern for climate change. I leave that in God's hands to not man's.\n\nHow would you express Christ to the world?", 'Climate change has stopped literally nobody from exploiting their oil. If it was supposed to be a ploy, it failed pretty fucking miserably.\n\nIncidentally, the advent of renewable energy has meant many developing nations with rich mineral deposits have benefited financially (when the US isn’t brazenly overthrowing their governments to get at it, of course). So your argument that it’s a plot to cripple developing nations is again baseless.\n\nCO2, methane, and other byproducts of human industry are targeted because they are the ones we are producing at a colossal industrial scale. Water vapour, unlike these gases, is not an issue because it turns back into water very easily and quickly: That’s where rain comes from.\n\nIt is estimated that humans currently produce 3% of all carbon emissions on Earth. That’s 3% more per annum than what nature was accomodating for before human industrialisation. If you think that doesn’t sound like a lot, run it through an interest calculator for a hundred years.', 'Can YOU fix climate change?', 'That is sad. Wondering how climate change affected it, thx for update', 'This FutureForward ads gives me chills.\n\nBecause climate no change.', 'Or maybe the climate change IS happening and we are starting to see visible symptoms through world wide drought?', "I'm leftish socially and politically. Have been all my life. Not so much so with the climate change catastrophe issue though. Canceling is definitely a thing. I'm surprised you haven't noticed it.", "Of course not. Okay, I should've been more specific: the air pollution (displayed as smog) shown in pic 1, is caused by airborne particles (visible), and climate change is caused by greenhouse gas (invisible). There are industries that do both at the same time, however, greenhouse gas is not airborne particles and does not cause smog, nor the airborne particles cause climate change, therefore these two parameters are conventionally counted separately when discussing the environmental impact. And overuse is not pollution by definition since it does not input any substance into the environment.", "Makes sense, Europe is turning off all it's nuclear plants.\n\nGiven the whole Russia thing, is insane.\n\nClimate change is real, but not caused by us at all, it is and always had been, the sun.", "Now you've gone from Pagan Earth worship to science worship. Just so you understand, science is just the process of testing and disproving. So because I disagree, makes me more scientifically minded than people like you who look at a chart and think it's the Bible. \n\nNothing you've provided proves me wrong.  You're looking at a scale of 10,000 years. A blip in geological time scale. Even a million years is. I also *never claimed that climate change isn't real.* I simply disagree that it's going to be catastrophic for the earth or the human species. You're also completely disregarding the fact that our weather is largely determined by the sun, which goes through cycles. \n\nCoral reefs are actually growing in many places. We have more trees now than 100 years ago. Plant yields will grow. Oceans are going to slowly, very slowly, rise. It will be so slow that it won't be a problem for people to relocate if necessary, but even still, plenty of countries (like the Netherlands) already exist well below sea level. \n\nNone of these climate models even consider human ingenuity or mitigation efforts. I'm much more concerned about over fishing, than I am about climate change. I'll be long dead and the temperature will have risen a whopping 2-3 degrees centigrade. \n\nFurthermore, these models are notoriously inaccurate. 50 years ago, we were all told that all of the evidence points to the Earth cooling and drying. They were telling people in GB to start planting succulents because it'll be so dry.", "Climate change is the fascism of weather. Anything I don't like can be attributed to it.", 'Beautiful, no climate change? I could meditate for hours', 'Fucking global climate change!', "&gt;it's workers that cause climate change. Nonworkers don't expand consumption because they don't expand production.", 'We have responded to COVID far more effectively than we have ever responded to climate change or are likely to (ozone layer is one counter example).', 'I mean, there have been a ton. \n\nI’d like you to explain what has happened to all the glaciers and permafrost if climate change isn’t happening?', 'I wonder which transportation methods are better for climate change than walking if you factor in food emissions.', "Va voir par toi-même: https://www.carbonbrief.org/analysis-which-countries-are-historically-responsible-for-climate-change/\n\nDescends jusqu'à Cumulative per-capita emissions et dans la partie gauche du tableau on est numéro 1 producteur de carbone divisé par la population.", 'For the most part, people stick to the same state but I’ve heard of people moving around state a lot. \n\nWith that being said, the 2 most common reasons I’d hear people moving for are work or change of climate', 'We have people who love bicycles, eventually change to cars. With our climate and distances, the bicycle is absolutely not popular. The average distance to work is about 15 km, the average travel time by public transport is about 1 hour. You can go once or twice a season. The rental of electric scooters is much more popular. But... with the general conflict of the very idea of 30,000 scooters (and a comparable number of cyclists who use a bicycle for trips) for 8 million cars, they do not make the weather.', 'yeah his commercial’s are very cringe.  but even aside from his cringeworthiness, this guy should not be anywhere near a public office *at all*.  he’s an outspoken christian nationalist since before the trump era (i think budd attended/graduated from a biblical seminary), he denies that human-caused climate change exists, he obviously opposes any sensible gun law reforms, he’s consistently voted against every single bill that would protect lgbtq+ rights (i think he’s said even in recent times that he still opposes marriage equality), he doesn’t want weed decriminalized, and he was/is in league with deranged traitors on january 6th.  \n\nvote this pig out of office.', "The miserable future that climate change will give us if you are don't change society or at least looking after for each other while facing the big drought and famines of the future", "Even if climate change was real, us eating bugs and using electric vehicles would do NOTHING as long as china continues to go on like that. But it was never about climate anyway. It's all about control.", "I wonder when Gavin Newsom is just going to send out packets of California Climate Change Kool-aid and tell everyone to drink it at the same time. It'll probably be on New Years.", "As a species, we made our choice regarding climate change decades ago.\n\nIt wasn't the right one.", 'Climate change', 'Die Summe wird sich wohl auf den CO2-Preis bzw. Carbon Offsets beziehen, sollte also als praktisch unerreichbares "Minimum" gesehen werden.\n\nHatte gerade noch [diesen](https://www.reddit.com/r/dezwo/comments/x2p1yq/jetzt_hilft_nur_noch_heiterkeit/) Artikel gelesen, welcher [diesen](https://www.reuters.com/article/climate-change-stimulus/tenth-of-pandemic-stimulus-spend-could-help-world-reach-climate-goals-study-idUSKBN271098) Artikel über eine Studie des Imperial College London zitiert, welche als Kosten zum Erreichen des Zwei-Grad-Ziel mindestens eine Größenordnung mehr nennt - aber immer noch nur ein Zehntel der Hilfsgelder in der Pandemie.\n\nEdit: Dummen Vergleich ausgebessert.', 'Number one driver of inequality and climate change.', "Your species isn't going to die. Well they might, but not from climate change.", "It looks like you shared an AMP link. These should load faster, but AMP is controversial because of [concerns over privacy and the Open Web](https://www.reddit.com/r/AmputatorBot/comments/ehrq3z/why_did_i_build_amputatorbot).\n\nMaybe check out **the canonical page** instead: **[https://www.texastribune.org/2022/05/18/climate-change-texas/](https://www.texastribune.org/2022/05/18/climate-change-texas/)**\n\n*****\n\n ^(I'm a bot | )[^(Why &amp; About)](https://www.reddit.com/r/AmputatorBot/comments/ehrq3z/why_did_i_build_amputatorbot)^( | )[^(Summon: u/AmputatorBot)](https://www.reddit.com/r/AmputatorBot/comments/cchly3/you_can_now_summon_amputatorbot/)", "I'm quite familiar with the topics of China, Russia, corporations, climate change, arable land (and land use more broadly), and the challenges of producing and distributing food globally. I'd say I have moderate knowledge of chemicals used in agriculture. I'd be happy to discuss this with you, but if you have no interest in discussing nobody is forcing you.", 'Pretty much every metric except for those related to pollution and climate change', '&gt; Klimakrise: Jetzt hilft nur noch Heiterkeit\n&gt; ---\n&gt;\n&gt; **Die Klimabewegung verbohrt sich in der Angst. Aber für Weltuntergangsstimmung haben wir keine Zeit.**\n&gt;\n&gt; *Von [Philipp Bovermann](https://www.sueddeutsche.de/autoren/autor-1.3421045)*\n&gt;\n&gt; 30\\. August 2022 - 6 Min. Lesezeit\n&gt;\n&gt; Das Beste und zugleich das Schlechteste, das man über die Aktionen der Protestbewegung „Letzte Generation“ sagen kann, ist, dass sie traurig machen. Die zumeist jungen Klimaaktivisten scheinen nicht mehr daran zu glauben, dass die Staaten die notwendigen Maßnahmen gegen eine katastrophale Erderhitzung ergreifen werden. Im vergangenen Sommer adressierten sie noch mit einem Hungerstreik im Berliner Regierungsviertel die Bundespolitik. Inzwischen ist ihr Gegner diffus geworden. Sie kleben sich an Autobahnauffahrten fest, an Gemälden. Am Wochenende versuchte eine Gruppe der Aktivisten, sich beim Bundesligaspiel des FC Bayern gegen Borussia Mönchengladbach mit Kabelbindern an die Torpfosten zu ketten. Bayern-Stürmer Thomas Müller, schon qua Name der Normalodeutsche schlechthin, rempelte einen der beiden an und brüllte ihm unhörbar etwas entgegen.\n&gt;\n&gt; **Herrje, können die sich nicht mal zusammenreißen?**\n&gt;\n&gt; Es wäre ein Fehler, die Klimakrise zu psychologisieren – sie ist keine Neurose, die Gefahr ist erdrückend real. Die Psychologie hilft aber, die Sackgasse zu erkennen, in die sich die Klimabewegung derzeit manövriert.\n&gt;\n&gt; In den vergangenen Jahren, seit Greta Thunberg ihre Wut und Verzweiflung in die höchsten politischen Gremien der Welt trug, begann eine Diskussion über die psychologischen Aspekte der anrollenden Klimakatastrophe, darüber, wie es sich anfühlt, auf einem angezählten Planeten zu leben. Meist war da von der „Klimaangst“ die Rede. Selten von der Trauer.\n&gt;\n&gt; Gegen die Angst empfehlen Psychologen das Prinzip der Hoffnung: Die „kognitive Dissonanz“ zwischen dem persönlichen Handeln und der immensen Bedrohung lasse sich nicht mit der Brechstange schließen, sagen sie, durch immer drastischere Warnungen. Damit die Angst nicht lähmend wird, sei es ebenso wichtig, sich zu vergegenwärtigen, was kollektiv und individuell noch alles möglich ist.\n&gt;\n&gt; **Das leuchtet ein: Wer nur Dystopien, aber keine Utopien anzubieten hat, überlässt anderen die Initiative und beschränkt sich aufs Verhindern. Man geht in die innere Sitzblockade**\n&gt;\n&gt; Die findet in den allermeisten Fällen aber nicht auf den Gleisen einer Kohletransport-Trasse statt, sondern in einem beheizten Wohnzimmer, auf dem eigenen, dicker werdenden Hintern.\n&gt;\n&gt; Der Verzweiflung zu entsagen, mutet skeptisch veranlagten Menschen allerdings auch einiges zu. Auf Tiktok geben sich viele junge Klimaaktivisten inzwischen betont optimistisch, handlungsfähig und „resilient“. Zum Beispiel der junge Mann aus Florida, der sich [dabei filmt](https://www.tiktok.com/@trashcaulin/video/7135814842878840110), wie er jeden Tag an den Strand geht und dort mit ganz viel Spaß Plastik sammelt. Das ist besser, als nichts zu tun, klar. Aber es schwingt darin auch der Verdacht protestantischen Kitsches mit. Luther wird das Zitat zugeschrieben, er würde, wenn er wüsste, dass morgen die Welt unterginge, „heute noch ein Apfelbäumchen pflanzen“.\n&gt;\n&gt; **Sollten wir, auch wenn morgen die Welt unterginge, heute noch unseren Müll trennen?**\n&gt;\n&gt; Der Treibhauseffekt würde ja sogar dann für lange Zeit weitergehen, wenn schon morgen kein Gramm Kohlendioxid mehr in die Atmosphäre gelangte. Die Katastrophe entfaltet sich bereits. Schrecklicher als die Angst ist nach Jahrzehnten verpasster Chancen womöglich die Gewissheit, was bereits unaufhaltsam verloren ist. Klimaaktivisten können heute nur noch versuchen, das Unausweichliche zu begrenzen.\n&gt;\n&gt; Vielleicht ist das, was da lähmt und zynisch werden lässt, daher gar nicht so sehr die Angst vor dem Kommenden, sondern es sind unbewältigte Gefühle: Trauer um die eigene jahrelange Unbeschwertheit, die den Kindern und Enkeln nicht mehr offensteht. Trauer, die sich wie ein Vanitas-Zeichen in die noch intakten Landschaften einschreibt. *All das wird irgendwann nicht mehr sein.* Trauer um die Menschen in anderen Teilen der Welt, die viel stärker unter den sich häufenden Dürren und Naturkatastrophen zu leiden haben. So viele von ihnen werden hungern, fliehen müssen und sterben. Wie soll man sich das als Teil der industrialisierten Verursachergemeinschaft verzeihen, gar optimistisch in die Zukunft blicken, während der eigene Lebensstil diese Probleme beschleunigt?\n&gt;\n&gt; **Am diffusesten bei all dem ist die Trauer um die Natur.**\n&gt;\n&gt; **Das Verhältnis zu ihr lässt sich deshalb so schwer fassen, weil große Teile der westlichen Geistesgeschichte aus nichts anderem bestehen als dem Bemühen, den Menschen aus ihr abzusondern.**\n&gt;\n&gt; Die verdrängte Verbundenheit mir ihr bricht nun, da sie zu verarmen und veröden droht, allenthalben als Wehmut wieder auf. Die Bücher und die Künste sind seit Jahren voll davon. In Bäumen, Pilzen, Moosen oder Kraken, überall erkennt der Mensch sich wieder, wanzt sich an die Natur heran und streicht ihr, wie zum Abschied, über die flauschige Wange. Oder aber er lässt sich auf ihr Gewimmel, ihre Amorphie ein, um auf feierliche Art sprachlos zu werden. „(...) und wo auch nur Pflanzen rauschen, klingt immer eine Klage mit“, schrieb der Kulturphilosoph Walter Benjamin.\n&gt;\n&gt; Sich von der Sprachlosigkeit überwältigen zu lassen, durch das Dickicht hindurchzugehen, darin liegt die Chance auf Heilung. Die irreparablen Schäden anzuerkennen, um die Zukunft wieder zu öffnen.\n&gt;\n&gt; **Dann, endlich, gibt es Hoffnung.**\n&gt;\n&gt; Politisch wäre es dringend geboten, diesen Schritt zu tun. Die Klimabewegung hat sich in den Bildern, die sie produziert, zuletzt immer tiefer in der Verzweiflung vergraben. „Fridays for Future“ nährte sich einen ekstatischen Sommer lang vom Glauben, man müsse den Leuten die Klimakrise nur nachdrücklich genug erklären („Listen to the science!“), dann würden sie schon losrennen und das „brennende Haus“, die Erde, löschen. Diese naive Hoffnung erzeugte einen ungeheuren gesellschaftlichen Schwung, überlebte aber die vielen unsichtbaren Widerstände in der Realpolitik nicht lang.\n&gt;\n&gt; Die Gruppe Extinction Rebellion machte kurz darauf auf sich aufmerksam als so etwas wie der dunkelromantische Wiedergänger dieser Träume. Die Aktivisten hielten „Trauermärsche“ ab, schminkten sich die Gesichter totenblass und lasen bei Sitzblockaden traurige Gedichte vor.\n&gt;\n&gt; **Auf ein Banner, das sie an der Westminster Bridge in London entrollten, schrieben sie: *"We\'re fucked".***\n&gt;\n&gt; Die „Letzte Generation“ erzählt nun ebenfalls vom Untergang, dieselbe Geschichte noch mal. Was soll nach ihr noch kommen? Die diesmal wirklich allerletzte Generation?\n&gt;\n&gt; Wo bleibt die Gruppe, die den Menschen die frohe Botschaft der ökologischen Befreiung von den Fesseln des fossilen Zeitalters bringt?\n&gt;\n&gt; **Wo bleibt die erste Generation einer neuen Welt? Es wäre eine bunte, laute Bewegung von ansteckender Heiterkeit, die sich nicht zu schade wäre, auch einmal taktisch fröhlich zu sein, wenn ihr eigentlich nach Weinen zumute wäre.**\n&gt;\n&gt; Die von der Krise direkt Betroffenen können es sich ja auch nicht gestatten, der Verzweiflung nachzugeben.\n&gt;\n&gt; Anstatt die Krise in immer noch und noch dunkleren Farben zu malen, den Berg des Unheils, den es zu erklimmen gäbe, immer weiter aufzutürmen, würden diese Aktivisten die planetare Krise vielmehr kleiner und beherrschbarer erscheinen lassen. Sie würden den Menschen sagen, dass es nicht nur, wohin man auch blickt, kapitalistisches Wachstum gibt, das zwangsläufig in den kollektiven Suizid führt, sondern schon heute Wege in eine bessere Zukunft, auf denen sie mitkommen können. Sie würden ihre Erfolge zu feiern wissen. Auch das Gute wächst ja, langsam und mit Rückschlägen, aber stetig. Grüne Technologien werden immer günstiger. Das Versprechen ist einfach zu gut: kostenlose Energie, die buchstäblich vom Himmel fällt.\n&gt;\n&gt; Noch vor einem Jahrzehnt ergaben Hochrechnungen, dass der damalige Kurs der Klimapolitik zu einem Temperaturanstieg von vier bis fünf Grad geführt hätte – die Erde würde gebacken wie eine Kartoffel. [Aktuell](https://climateactiontracker.org/global/temperatures/) steuert die Welt auf ein Plus von 2,5 bis 2,9 Grad zu. Die Auswirkungen wären katastrophal, aber doch etliche Höllenkreise weniger schlimm.\n&gt;\n&gt; Den Rest zu schaffen und die Erderhitzung auf „deutlich unter zwei Grad“ zu begrenzen, wie im Pariser Klimaabkommen festgehalten, ist keineswegs unrealistisch. Im Oktober 2020 [errechnete](https://www.reuters.com/article/climate-change-stimulus/tenth-of-pandemic-stimulus-spend-could-help-world-reach-climate-goals-study-idUSKBN271098) ein Team von Wissenschaftlern des Imperial College London, dass dafür schon ein Zehntel der zwölf Billionen US-Dollar, die damals als Corona-Rettungspakete versprochen waren, reichen würde. Möglich, dass die Annahmen etwas optimistisch waren. Aber die Rettung der Welt wäre jedenfalls nicht unerschwinglich.\n&gt;\n&gt; Dass es nicht passiert, liegt an ökonomischen und politischen Interessen. Der Logik der Macht begegnet man ebenbürtig nicht in der Logik der Verzweiflung, indem man aus gefühlter Ohnmacht, die Mächtigen treffen zu können, wahllos Konsumenten-Fußvolk gegen sich aufbringt und die Gesellschaft in der Klimafrage weiter spaltet. Den Planeten und seine Bewohner zu schützen, ist unterm Strich auch nur ein Interesse, das es strategisch durchzusetzen gilt. Anstatt Verzicht zu predigen und ständig zu betonen, warum das alles sowieso nicht reicht, sollte man den Menschen etwas versprechen – *fake it till you make it*. Die Gegenseite tut ja auch nichts anderes, nur viel ungenierter.\n&gt;\n&gt; Vielleicht ist es, so besehen, gar nicht schlecht, beim melancholischen Grübeln zum Geräusch der tickenden *Doomsday Clock* etwas zynisch geworden zu sein. Den Weltuntergang zu verhindern, einfach so als Selbstzweck, zieht nicht? Na, dann bindet man in drei Teufels Namen eben auch noch eine rosa Schleife darum.', 'This is happening *because of climate change* and those regulations on farmers are trying to *slow climate change*. \n\nWhat about that is hard to understand?', 'No :)\n\nChina, Russia, Corporations, Climate Change, useable land, feeding 7 billion people, getting food to 7 billion people, the chemicals we farm with production, use, and sale.\n\nI could keep going on but i have no further inclination to speak with you. Have a great day.', 'It\'s amazing how much wrong someone can pack into one paragraph.\n\nThere is a huge amount of uranium available from multiple suppliers, it can also be filtered economically from seawater because it\'s so energy-dense, and there are other materials which can be used, including thorium. There is effectively zero "scarcity" of uranium and there won\'t be for many decades, if ever. It is *not* running out.\n\nClimate change hasn\'t shutdown "most" of the reactors in Europe : it\'s reduced the output of 4-5 reactors in France which are waiting for the rivers to fill up again, and the remaining reactors in France are in maintenance and will be back online by the end of the year. [Here\'s the graph](https://i.imgur.com/QvnWdgp.png) of their return to service.\n\nI\'ve just counted : literally every sentence you uttered was wrong. Extraordinary.', "Its not science to me. For me, its you that is the Flat-Earther. Nature responds to change. I doubt science can predict this. For example, it may rain more due to warming.\n\nThe above said and returning to the OP, Buddhism teaches the social world arises from ignorance &amp; craving (SN 12.44); therefore, as the population of ignorant &amp; craving people grow, we should prepare for the worst. In other words, just let it go.\n\nTo conclude, I correctly advised if you believe in climate change, you should discipline yourself by living a sustainable life. This disciples is a step on the spiritual path to freedom from anxiety.\n\nBut if you are a hysterical untrained undisciplined young person, addicted to entertainment, addicted to TV, addicted to NetFlix, addicted to your I--Phone and Internet, addicted to wearing make-up, etc, then you really won't have any coping skills when your beloved mass-media &amp; Greta keep telling you there is a climate emergency.\n\nThe proof is in the pudding buddy. Are you another upper-middle class Climate Warrior on large income, with 4WD, flying around the world on nature holidays, etc,?\n\nIn Buddhism, we overcome anxiety by practicing the Path, which includes Renunciation of sensual/material indulgences as its second factor.\n\nThat's it. Bye bye.", "We poor and middle class members are the ones voting to ignore the climate change, so kinda deserved. The rich are a tiny minority in a democracy, and lobbyism/corruption doesn't change that.", "Antinatalism - Explosive population growth in the last century. Humanity in ever-burgeoning numbers are destroying the globe's ecosystems and the trajectory of human-induced climate change is ultimate annihilation.\n\nHowever, it seems to be morally corrupt to even raise a question about what unrestricted procreation is doing to our planet. Antinatalism as a philosophy is off the table, a non-starter.\n\nBut what would our world be like now if the population were 1.6B as it was in 1900, instead of almost 8B as it is now?", "Even assuming climate change is real and 100% human caused that's not reason to grind civilization to a hault and implement all of this crazy statist taxation schemes.", "&gt;I've heard suggestions that nuclear winter would be a way to shut down global warming.\n\nThe problem is, CO2 lasts for at least 1200 years, not to mention all the other gasses, and the burning of hundreds of urban targets and billions of bodies will only add more gasses into the atmosphere. The thing about nuclear or volcanic winters that causes the cooling effect is the soot or ash particulates, and these drop back down to the ground pretty quickly, say within 10 years.\n\nSo the actual effect of nuclear winter will be a massive and sharp increase in emissions with a very short term hiatus and then the newly added emissions will send climate change in overdrive!", 'See, it\'s like this (or so I\'ve learned from the electric car and climate change shills):\n\nElectric cars are "green" and so are lithium batteries (bec they\'re used in "green" electric cars); therefore, carbon emission during "green" lithium mining is, by default "green".\n\nTa da!\n\nEdit: grammar', 'Is it weird that even though south park is supposed to be the more political witty show....freaking family guy of all shows may have the better takes?\n\nSouth park spent a decade denying climate change, family guy was all in on pro choice episodes and such', "&gt; Couldn’t action on climate change be how god prevents us from destroying the planet?\n\nThen he probably shouldn't have opposed action on climate change.", "The following submission statement was provided by /u/-AMARYANA-:\n\n---\n\nSS: Citizens around the world can sense that something is very wrong, and it looks like the economy may be headed for a serious recession in the near term. The operation of our economy requires energy of the correct type and the right quantity. The International Energy Agency and politicians around the world have recommended a transition to the use of renewables to try to prevent climate change for quite a few years.\n\n---\n\n Please reply to OP's comment here: https://old.reddit.com/r/Futurology/comments/x2nyfg/the_worlds_energy_problem_is_far_worse_than_were/imkjz23/", 'Esta personagem - o Dr. S. Matthew Liao - também acha boa ideia transformar os nossos olhos em olhos de gato para consumirmos menos eletricidade à noite, e tomar comprimidos que nos fazem ter pena do planeta. [Fonte](https://www.marketplace.org/2016/12/15/five-ways-bioethicist-wants-change-our-bodies-fight-climate-change/)', 'Climate policy changes have reduced projected warming from &gt;4c to ~3c by the end of century\n\n\nhttps://nitter.kavin.rocks/hausfath/status/1511018638735601671#m\n\n\nhttps://climateactiontracker.org/\n\n\n&gt;ending 99.9% of humanity\n\n\nNo reputable scientist predicts that even under worst-case scenario(rcp8.5), not to mention the current one(rcp4.5)\n\n\nhttps://climatefeedback.org/claimreview/prediction-extinction-rebellion-climate-change-will-kill-6-billion-people-unsupported-roger-hallam-bbc/\n\n\nhttps://www.carbonbrief.org/in-depth-qa-the-ipccs-sixth-assessment-on-how-climate-change-impacts-the-world/', "I always suspected this. When the meteorite hit that wiped out the dinosaurs, the climate didn't change 'slowly'.\n\nOn a geological timescale, we are a literal meteorite.", 'Do you think you need to see pollution for it to be pollution? What causes climate change?', "Which lies are you referring to on the left?  I can't think of any prevalent ones.  Please don't make me dig out a link to the 2020 GOP-chaired SIC report on Russian election interference.\n\nClimate change hoax, plandemic, stolen election, etc.  I know not every GOPer thinks these things, but a frighteningly large percentage do, and the mainstream right is careful not to alienate those people.  You literally have to garner support of stupid conspiracy theorists to win a GOP primary these days.  If I were a reasonable conservative I'd probably switch parties and vote for center right dems in primaries.  Better that than morons like Boebert.", 'Climate change.   Oops...too late.', 'I actually can see this\n\nSeth mcfarlane is very clear which sides he is on, he is a registered democrat who has made pro abortion chapters and such\n\nMatt stone and trey parker are more lolbertarians who denied climate change as far as 2006 and literally  made the meme that became emblematic of enlightened centrism for the 2004 election', "Elon is right. But it'll be due to climate change and starvation.", "Hopefully be taken care of by living in an actual community. I suspect that community is bound to re-emerge at some point in the near future, if only in response to drastic climate change, the collapse of globalization, and the escalating cost of everything. \n\nI mean, or just die having upheld our current notions of complete atomization, alienation, and separation from human community and the larger community of life. That's pretty possible too.", "Bruh. BRICS ain't going to do shit. Climate change is going to SERIOUSLY start to bite pretty soon and there's not enough cheap energy left in the world to project power across the globe. Can't gun for world domination when you have a revolution at home.", "&gt;I'm not particularly concerned about government employees actively grooming children to convince them to mutilate themselves, no\n\nOk, then I guess I don't really feel the need to convince you. \n\n&gt;climate change and greenhouse gas emissions\n\nthat's literally just a smokescreen for the malthusians.", "I always love seeing folks' passion projects like this! And this one is certainly in a league all its own when it comes to effort and detail.\n\nThere are some routings I wouldn't necessarily have chosen, though I see why they were, and some I would add, but overall it's a solid pitch. I haven't read through the report yet, so no idea on great/good/bad/ugly there, but a quick scroll through looks very well done (as is to be expected from a professional).\n\nOne thing I'll say that I don't like, and this is mostly angled at some of the comments on the Urbanize page, is that I always *hate* pitching this kind of proposal for wider regional rail *against* other modes of transit. In this case heavy and light rail.\n\nThere's more than enough pent up demand for transit-oriented (and walkable and bikeable and dense and etc.) development, and necessity to meet the challenge of climate change, to justify a wide, combined network, something that the paper seems to recognize with infill stations, (modest) service expansions for existing heavy rail, and the integration of light rail / streetcar. I've always preferred a more expansive vision (See: https://www.getonboardgeorgia.org/vision), but I don't disparage someone from keeping their efforts focused. What I do disparage are people who say 'do THIS instead of THIS OTHER THING!'. Please. Just stop.", 'https://www.weforum.org/agenda/2021/07/why-we-need-to-give-insects-the-role-they-deserve-in-our-food-systems/\n\nhttps://www.weforum.org/agenda/2022/02/how-insects-positively-impact-climate-change/\n\nhttps://www.weforum.org/agenda/2018/07/good-grub-why-we-might-be-eating-insects-soon/\n\nhttps://www.weforum.org/videos/21336-europe-has-approved-insects-for-human-food-uplink', 'Someone said there was a climate change sign somewhere', "Fly all over for useless conferences. It's certainly making climate change better", 'At least California cares about doing something about climate change and protecting the planet unlike other doltish science-denying states', "As a wise sage said: They're doing their best to collect as much data as possible, but it's really difficult at the moment because all the mice seem to keep dying from climate change before they can make definitive observations.", '&gt;most of that money went to climate change projects\n\nNo it didnt.  Go ACTUALLY look at what\'s in it. Here\'s to help get you started.\n\n[https://www.whitehouse.gov/briefing-room/statements-releases/2022/08/19/fact-sheet-the-inflation-reduction-act-supports-workers-and-families/#:\\~:text=The%20Inflation%20Reduction%20Act%20will%20protect%20Medicare%20recipients%20from%20catastrophic,for%20the%20first%20time%20ever](https://www.whitehouse.gov/briefing-room/statements-releases/2022/08/19/fact-sheet-the-inflation-reduction-act-supports-workers-and-families/#:~:text=The%20Inflation%20Reduction%20Act%20will%20protect%20Medicare%20recipients%20from%20catastrophic,for%20the%20first%20time%20ever).\n\nWe live in a country where businesses and oligarchs have more power and influence over policy than the people or politicians do.  The government is restricted from what they are able to do to businesses.  They\'re in control.  Nothing about that is "moderate".  Compare our capitalism to any other country, and you\'ll see that our capitalism is far more broken than any other country. \n\nI\'d love to hear your explanation why record inflation is happening at the exact same time that companies are posting the highest profits they\'ve had in many decades', 'Wonder why……………….. is the current explanstion climate change?', "The Daily Wire doesn't accept reality.\n\nParticularly with regards to climate change.", 'Ethiopia’s Corbetti geothermal plant have plans for 1000MWe. 100 trillion in stranded assets if we get off fossil fuel and address climate change. \nInfinite growth paradigm, finite resource replenishment. No regard for biosphere inhabitability, soley infinite cyclical consumption and accruement of capital/profit. More debt in existence than actual currency.', 'Well, in the context of the story that scene is definitely a commentary on the role of women in medieval societies and their lack of autonomy, choice and subjugation by the patriarchy. Since Martin’s work often time contains allegories to modern issues like climate change, it wouldn’t be far fetched to connect this scene of a woman’s reproductive choices being violently taken from her to the modern issues of abortion and feminism.', "Hot Summer = Proof of Climate change \nCold Winter  = Local weather doesn't matter", 'The billionaires need to give up their lavish lifestyle that’s what’s really affecting climate change.  Done believe any propaganda about the common man needing to change anything it’s all a bunch of BS', "Not having kids is not a solution in any ecologists plan to tackle world hunger or climate change. It's global policies and investment in renewable energy sources as well as environmental stewardship education.\n\nYou know what those need to work? Future generations.", 'Future where the US receives all of the climate change and Australia gets scrambled.', 'It started with the vaccine and it’s massive. The only explanation is either climate change, which is the story the media is going with, or the vaccine. Anyone with any neural synapses knows what’s up. \n\nSeriously try and keep up.', 'But at least they probably would join protests for awareness about climate change , right…effing pigs', "Because it's forking natural, look at history, there's been atleast three ice ages then everything melts then it gets hot as fork and most species die and a few survive then everything starts freezing and ice comes back new ice age then it melts and repeat\n\nEveryone doesn't do research before posting or saying things 40 years ago in the 80's they were saying prepare for the ice age and now they're preaching climate change, it's real and natural and inevitable because you can't beat nature it always finds a way and it's just a way for people to make money", 'you can read a bit more about [jellyfish blooms](https://www.businessinsider.com/jellyfish-thriving-climate-change-warm-oceans-2019-10?amp) here!', 'How do you feel about the impacts of climate change on future generations?', 'No it’s still an issue but you can’t attribute all ice melt or shearing to climate change. It’s a normal process. Orcas breaking ice while hunting is not a new phenomenon.', 'Climate change is "political" in the same way that covid or gay people existing is "political"', 'Exactly. The profit motive rarely incentivises the right thing, for example climate change.', "There was a taboo about Nuclear maybe in no small part because of misinformation for a long time because of the potential for fallout and such. Reading into it though the fact that there's been a significant amount of improvements means there really needs to be a reevaluation of the issue expecially since Climate Change has become a much more serious issue in recent years and because while Nuclear has it's risks it doesnt contribute to Climate Change since it doesnt produce CO2.", 'We’ve all been staying indoors for decades here in Phoenix in the summer. Just like people in North east do when it’s winter time. We love the summer. I’m all about the pool season and camping is 1.5 hours north in the cool pines. People that don’t live here always talk about how terrible and doomed we all are, but you need to experience it to know. I’ve lived here almost 40 years. I’ve seen the climate change. It’s there, but if the southwest is truly doomed in 30 years, so is most the rest of this country (and world). I just don’t play into this forecasting for the future stuff.', 'It is a climate change', "„ Concerns about climate change or earthquakes, when mentioned, were framed as positives rather than negatives.[16] Sörgel's 1938 book Die Drei Grossen A has a quote from Hitler on the flyleaf to demonstrate that the concept was consistent with Nazi ideology.“\n\nBig surprise /s", 'With Climate Change staring us in the face, I\'m worried that we may one day be calling these the "good times".', 'Praying is stupid, we need to stop climate change.', 'I think it’s an attempt to make climate change scarier to the layperson by blowing an insignificant statistic out of proportion. All the while increase in the heart attack rates are obviously caused by the obesity epidemic.', 'It can’t be a prequel anymore since *FAM* has solved climate change.', 'And people are gobsmacked that we can’t manage meaningful climate change goals, even moderate ones.', 'There is one for climate change. It reduces the number of humans.', 'This thread conflates what I see as two different questions: Why I personally didn\'t have children vs. why I now think no one should have children.\n\nI\'m 68 years old, so my reasons for not having children some 40 years ago were far more personal than ideological. I just didn\'t want them enough to be worth the incredible effort and expense of raising them. \n\nOver the years, however, as population continues to climb and climate change accelerates, I\'m convinced that having children is just not a good idea for anybody in the developed world. Every individual in the "first world" puts too much pressure on resource and energy use, not to mention that the world they\'re going to inherit will be collapsing around them.\n\nWhen I was born there were 2.5 billion people on Earth. We\'re approaching 8 billion people now. Enough.', "&gt; in the big picture of 100's of millions of years.\n\nI don't know how many 100's you're talking about, but modern humans have only been around for roughly 1/3 of a million years.  \n\nYes the climate has changed greatly over the lifecycle of the earth, and just about 65 million years ago it was uninhabitable to almost all life.  This is irrelevant in any conversation about the modern day effects of climate change.\n\n\nIn other words, wtf are you talking about?", 'Can confirm the lemon juice trick works. I did it myself last year when it was going on. \n\nSide note: I wouldn’t be surprised if this was a reoccurring thing each fall in Btown going forward. Algal blooms are a likely source of both compounds, and with those increasing in frequency due to climate change, I doubt we’re out of the woods after this year.', "The following submission statement was provided by /u/IHathReturned:\n\n---\n\n&gt;In October, the Intergovernmental Panel on Climate Change (IPCC) released a report setting out why we must stop global warming at 1.5\u2009°C above pre-industrial levels, and how to do so1. It is grim reading. If the planet warms by 2\u2009°C — the widely touted temperature limit in the 2015 Paris climate agreement — twice as many people will face water scarcity than if warming is limited to 1.5\u2009°C. That extra warming will also expose more than 1.5 billion people to deadly heat extremes, and hundreds of millions of individuals to vector-borne diseases such as malaria, among other harms.\n\n&gt;But the latest IPCC special report underplays another alarming fact: global warming is accelerating. Three trends — rising emissions, declining air pollution and natural climate cycles — will combine over the next 20 years to make climate change faster and more furious than anticipated. In our view, there’s a good chance that we could breach the 1.5\u2009°C level by 2030, not by 2040 as projected in the special report (see ‘Accelerated warming’). The climate-modelling community has not grappled enough with the rapid changes that policymakers care most about, preferring to focus on longer-term trends and equilibria.\n\nThe IPCC is conservative in its predictions. This paper, published in 2018, details the reasons to expect global warming to occur faster than expected.\n\nThis is related to collapse because global warming on its own can destroy the food system through floods, droughts impacting food production.\n\n---\n\n Please reply to OP's comment here: https://old.reddit.com/r/collapse/comments/x2n19l/global_warming_will_happen_faster_than_we_think/imkeimo/", "In the South, schools refuse to teach things like CRT and climate change. For the sole purpose of dumbing students down, in hopes they'll grow up Republican.", '&gt; Democrats actually consult with experts in the field before they craft legislation.\n\nSo what expert determines that a rifle with the ability to attach a knife is magically more deadly than the exact same rifle without that ability to attach a knife because a bayonet lug is a feature that creates an assault weapon.  But to my knowledge there is not a problem of people being stabbed by rifle mounted knives.\n\nAlso I would love to find the expert that said "A heat shield exists purely to prevent the shooter from burning their hand on a hot barrel.  Obviously by banning this safety feature we will save lives."\n\nAlso where are these experts that say a rifle with a permanently collapsed stock is ok and one with the same stock permanently extended is ok but the same rifle with the same stock that can simply move between fully collapsed and fully extended, when the rifle in either configuration is still legal, somehow now is a weapon of war only suitable for armies and local police...\n\nGo ahead find me these experts.  They probably came from the same schools that create the "experts" republicans use to deny climate change\n\n&gt;Also, Presidents don\'t write laws.\n\nOh ok so then every time Trump said something dumb and everyone pointed out his ignorance all Republicans had to do was say he doesn\'t write the laws and then magically his ignorance is entirely unimportant?', "I would be incredibly surprised if any vehicle company that does business in the US and doesn't exclusively make EV's hasn't thrown plenty of lobby money against climate change laws.", 'it’s causing environmental factors that can cause heart attacks, such as wildfires, droughts, and heatwaves. Makes sense to me 🤷\u200d♂️ doesn’t mean it’s the main cause, but it’s just more reason to be concerned about climate change. P', "Unexpected blips will happen more and more with climate change. Are we just going to have a un ending homeless population? That's not the kind of country i want to live in", 'All climate change stuff.', "&gt;How could we not think that this was unsustainable?\n\nThe same way we all though climate change wouldn't get us, or that the economy would just keep going. When things are good, people don't *want* to think", 'The same SF who have no policy at all on climate change? Its almost as if they are short term populists with no idea how to actually govern. All the young folk dying for a SF government should take a good look at the actual implications of their policy positions (or lack thereof).', 'We are beyond the point of a competency test for voting,  Look all I\'m saying is the people who believe the Earth is flat and the people who look at climate change and they don\'t think "oh yeah that\'s just grade 10  Newtonian physics"  Not only do they get a vote but their vote is equal everybody else\'s.\n\nEdit addition : we have literally watched the federal education budget fall flat on it\'s face since the early 70\'s and subsequently the military\'s discretionary budget increase and no ome bats an eye', "Climate change? That is code for UN Commissars telling us Americans what temperature it is going to be in our outdoors. I say let the world warm up. See what Boutros Boutros Ghali Ghali thinks about that. We'll grow oranges in Alaska.", '*oof*… that chart on different countries’ political ideologies on climate change. The GOP does laps around other right wing parties. Literally every other countries’ conservatives (besides Israel, who still are much higher) are over twice as likely to believe in climate change', 'Do you even know what climate change is', "It's very telling how you link to the appendix, instead of the fulltext. \n\nhttps://www.thelancet.com/journals/lanplh/article/PIIS2542-5196(21)00251-5/fulltext\n\nOtherwise people might have a chance to read the authors' actual conclusions:\n\n&gt;Compared with the cost of current diets, the healthy and sustainable dietary patterns were, depending on the pattern, up to 22–34% lower in cost in upper-middle-income to high-income countries on average (when considering statistical means), but at least 18–29% more expensive in lower-middle-income to low-income countries. Reductions in food waste, a favourable socioeconomic development scenario, and a fuller cost accounting that included the diet-related costs of climate change and health care in the cost of diets increased the affordability of the dietary patterns in our future projections. When these measures were combined, the healthy and sustainable dietary patterns were up to 25–29% lower in cost in low-income to lower-middle-income countries, and up to 37% lower in cost on average, for the year 2050. Variants of vegetarian and **vegan dietary patterns were generally most affordable**, and pescatarian diets were least affordable.", 'You might wanna check your legal obligation if you are renting your flat. \nBelow 16 degrees, molds and other stuff could happen and damage the appartement. Which I believe you could be accountable for. \nBest idea would be to try not going below 16.\nBut then… will we have less than that this winter seeing the climate change', 'But yeah, you should listen to the people telling you climate change is faked by China', "In 2010 the City budget for all property taxes was $1B.  Now is it just over $2B - that's a increase of over 100%.  Works out to compound increase of 6% per year.  That's way more than inflation over that period of time.  The MVA system hides it well.\n\nOperating costs of the City have more than doubled and the pace of increase is on track to double the operating costs again by at least that much in the next decade.  Throw in climate change spending and the 19+ new communities being built out and the picture for property taxes is very ugly.", 'all of our opinions probably don’t even really matter, because we’re in a mass extinction event and soon enough we’ll be fighting over water thanks to climate change, sooner than we think!', 'Climate change should not be a partisan issue', "I don't have anger at politicians because of climate change, I'm angry that politicians inflict harsh economic conditions on their own people in the name of climate change.  Save the planet, freeze and starve the people.  Save the planet by charging us twice as much for gas.  Save the planet by making sure farmers can't grow food.  Save the planet by committing to irrational co2 emission standards while allowing China to run rampant with them - hello, we all share the same planet dumbass! Save the planet by blithely telling us to get electric vehicles when many can't even afford rent and groceries. Save the planet by telling us to eat bugs. Speaking for myself I'm not angry because of climate change anxiety, I'm angry over what's being done to us under the guise of climate change.  And you're one of the worst, Trudeau.", 'But that Jim Inhofe held a snowball in the chambers of congress! Climate change is NOT real. /S.', 'And you can bet most of them are middle class lovvies living off mummy and daddy who protest for climate change.', 'Bernie considers climate crisis the most essential change to national security. He opposed the establishment of the Dakota Access Pipeline on the grounds that it will have a huge effect on the climate.\n___\nBernie Bot -Bernie_Bot_02', "People still are not believing that climate change is real.  I hope you examine scientific evidence as to what climate change is and how it will impact you.  You're serving the disinformation campaigns from Oil industry.", 'More than likely the river backed into the spring due to flooding. I was at Wakulla recently and it was extremely clear. That being said, climate change leads to more frequent brown outs of springs and can alter spring chemistry (through an increase of storm frequency and intensity). Source: been there, and I’m a hydrologist\n\nEdited to add when the water table is low, water from other conduits will be pulled in the direction (high to low hydraulic head) of the spring and can cause more tannic water to come out of the spring, but this will vary depending on aquifer levels. We see that with some of our springs here near Gainesville that sit at a higher potentiometric elevation.', "And also it's making it seem like electric vehicles are the problem when the fucking heat waves and climate change are the problems.", "did you see the 4 or 5 pages of imgur l linked.. \n\n individual links \n\n https://i.imgur.com/5PUpk6p.jpg\n\n https://i.imgur.com/eSnAPR0.jpg\n\n https://i.imgur.com/Ap8qG67.jpg\n\n https://i.imgur.com/NdBbVvF.jpg\n\n  the EGR solenoid is a bleed type... when not engaged it bleeds air.. so the vacuum supplied is always leaking..  the computer pulses the EGR solenoid to change the amount of opening of the EGR valve..  so tee'd into the red hose to the EGR solenoid you probably will get less vacuum..\n\n  and switching to defrost when driving uphill is SCREAMING at me VACUUM LEAK some place..  i have been at this for 43 years as a professional... i can't tell you where its leaking.. but i tell you you have an issue.. could be between the HVAC vacuum reservoir and the climate control head... \n\n save and print those diagrams...", "The inflection point. Energy is at a focal point thanks to sanctions on Russia, and historic global droughts impacting energy production while highlighting climate change. There's been so many crises throughout my short life, but this one's a kick on the arse if there's ever been one.", "Yes, we should be altering the path and deploying more clean energy. (I'm not claiming JK is interesting in that, although I believe EM is.) But there's no scenario where a great deal of short-term human suffering is avoided without fossil fuels.\n\nI'm expecting this winter to be really quite bad for much of the world. Maybe the winter after that too? But certainly this winter. Entire industries are shutting down in Germany because they lack affordable electricity and chemical inputs.\n\nAnd that's WITH Germany buying up all the world's LNG. Developing nations can't afford to import it any more. Very destabilizing. I don't think the 2022 impact of climate change is going to stack up to the impact of 2022 energy-crisis / fertilizer-crisis.\n\nMaybe JK doesn't really care about GW, I don't care. Broken-clock being right twice a day. On this point JK regurgitating EM is regurgitating true statements.", 'Ooh I\'ve got some:\n\n* Letting motorcycles filter between cars saves lives.\n* Daily bathing is a scam concocted by Big Soap. (Seriously, look it up).\n* Women shaving their legs is a scam concocted by Gillette Razors. (Again, look it up).\n* The USA is not a capitalist, democratic state. It is at best an oligopoly, if not a straight up oligarchy.\n* "Socialism" the way Westerners use it is a misnomer. (They inevitably mean "authoritarianism").\n* Climate change is real and humans are accelerating its course. We\'re all about to have some serious egg on our faces in the next couple of decades.\n* News in the US is horseshit, period. Not "left side" or "right side" (although IMO the right is more blatant about not caring that it\'s horseshit). But compared to other countries, US citizens don\'t know fuck all about what\'s going on in the world.', '&gt;In October, the Intergovernmental Panel on Climate Change (IPCC) released a report setting out why we must stop global warming at 1.5\u2009°C above pre-industrial levels, and how to do so1. It is grim reading. If the planet warms by 2\u2009°C — the widely touted temperature limit in the 2015 Paris climate agreement — twice as many people will face water scarcity than if warming is limited to 1.5\u2009°C. That extra warming will also expose more than 1.5 billion people to deadly heat extremes, and hundreds of millions of individuals to vector-borne diseases such as malaria, among other harms.\n\n&gt;But the latest IPCC special report underplays another alarming fact: global warming is accelerating. Three trends — rising emissions, declining air pollution and natural climate cycles — will combine over the next 20 years to make climate change faster and more furious than anticipated. In our view, there’s a good chance that we could breach the 1.5\u2009°C level by 2030, not by 2040 as projected in the special report (see ‘Accelerated warming’). The climate-modelling community has not grappled enough with the rapid changes that policymakers care most about, preferring to focus on longer-term trends and equilibria.\n\nThe IPCC is conservative in its predictions. This paper, published in 2018, details the reasons to expect global warming to occur faster than expected.\n\nThis is related to collapse because global warming on its own can destroy the food system through floods, droughts impacting food production.', 'Climate change', "Depending on the sciences you have differents name like the people that don't belive in climate change are climato septic the people that don't belive the earth is round are flat earthers the ones that don't like needles but are too afraid to say it are anti vaxxers etc people that don't belive in evolution and the concepts that go with it like baze are creationist", "Has there ever existed a time in the history of the world that one place wasn't extremely hot, anther place wasn't flooded, another wasn't experiencing a drought, when ice wasn't melting somewhere and where there were no fires?   \nThe climate has always been changing and that will never change. However your ability to see different parts of the world in real time has certainly changed over the last 20 years. And that's what's got you confused about what's normal and what isn't. What was distal is now proximal, and confusing the heck out of your brain when it comes existential risk. Your world is perfectly ok. Yes weather is changing, always has, and that has always been a threat to humans living in those areas where it changes inhospitably. \n\nWhile you may be scared of the weather, is it not scarier to believe that some people think they can control it? Can you control the weather?", 'Nah, honestly as much as I don\'t want kids for reasons outside of my health issues (climate change, global issues, my mental health, etc) I\'d take being able bodied and having a child over having a progressive incurable genetic disorder that leaves me in constant pain and is making my life super shitty and will only make it shittier as time goes on. Albeit this is a better scenario than if I had let my exe\'s family bully me into a kid, as then I\'d be a disbled single parent who passed on a genetic disorder. But "best possible scenario"? Far from.', "wasn't that about climate change not covid?", "Put all stores on stilts, that way you can't drive a car into them. \n\nWould be great for avoiding flooding, you know, climate change and all.", '/u/PerBolund \n\nFossila bränslen kan inte helt ersättas av förnyelsebart förrän på 2040-talet enligt [Energimyndigheten](https://www.energimyndigheten.se/nyhetsarkiv/2019/sa-kan-100-procent-fornybar-elproduktion-se-ut/). Det betyder att vi har en övergångsperiod på minst 18 år. Vad gör vi under dessa 18 år?\n\nMP säger förnyelsebart, trots att det finns inte på bordet förrän på 2040-talet. Det svaret är alltså inte acceptabelt. Fram till dess har vi endast 2 alternativ:  \n\n* Använd minimal mängd kärnkraft, och maximal mängd fossil energi\n* Använd maximal mängd kärnkraft, och minimal mängd fossil energi\n\nNi tycker alltså att det är bättre för miljön att bränna gas och kol i 20 år, än att använda kärnkraft. Anmärkningsvärt dåligt påläst. [Som tur är så är medelsvensson bättre påläst](https://www.analys.se/opinion/) .\n\nJag uppmuntrar alla att titta på den här dokumentären, "[Do we need nuclear energy to stop climate change?](https://www.youtube.com/watch?v=EhAemz1v7dQ)" av Kurzgesagt. [Kan man lita på Kurzgesagt?](https://www.youtube.com/watch?v=JtUAAXe_0VI)', 'Well my point is, it’s not that unbelievable if it were real. This seems very much on par with things.\n\nMake money in a way that sells you and your property as ad space instead of meaningful work✅\n\nThe ad is for a objectively evil company✅\n\nThe ad promotes selling water which said company argued that water isn’t a human right✅\n\nMaking it a bumper sticker for a car, my car, that did help to contribute in some small way to climate change making water scarcity a thing✅\n\nEtc, when I saw this it definitely felt extremely fucked up.', 'Pollution is also in the air, which is causing the atmosphere to heat up along with weather patterns to change. Admittedly, a lot of that lower water probably has to do with over use, but climate change from pollution is a contributing factor', 'They tell me its because of climate change.', 'Is there a verse on climate change I’m unaware of?', "&gt; climate change makes food next to impossible to grow in most places\n\nuh... that's not a real climate change outcome at all.", 'While I’m glad that hurricane season is quiet and hope that it remains that way, I’m concerned that if it remains quiet, republicans before the midterms can say that the climate scientist’s wrong prediction of an active season is “proof” that climate change is hysteria pushed by democrats.\n\nEdit: It also doesn’t help that the US has been relatively spared by severe climate disasters this year compared to the rest of the world.', 'Quoting a conservative think tank that denies climate change seems about right for someone with opinions as deluded as yours', 'Yes we have. Before climate change it was "global warming", before that it was "acid rain", before that it was "next ice age", before that it was "the hole in the ozone layer".\n\nWe\'ve been dealing with a forever collapsing climate for centuries. London had a mini ice age that lasted for a few years and then had a few years of extra hot weather. We have actual records of these things happening.\n\nClimate collapse is something we\'ve been flip flopping on for decades.', 'SS: you can how Elon Musk is really scared of climate change. While he praises the number one emitter and flys private jets', "What about climate change and the eventual climate collapse? We haven't dealt with that before.", 'Honestly, at this point, climate change deniers deserve hell. How could you? I wasn’t an avid climate change believer in that I knew it exist since I was like in Middle school learning about it in school. Until each year, I’d see the figures people claimed and warned about years ago, actually exceeding what was expected. Each year summer gets hotter to dangerous degrees. I live in the northeast and literally people can’t live. People are sweating 24/7 in dangerous extends, the subways are full of sweats, and i don’t think we can bear the uncomfortableness much more. This isn’t about ‘Liberals’, this is science. Even religious scholars, atleast Christians anywhere but America, Muslims, Jews all believe the value of taking care of our planet and the destruction society is causing it cus we are actually facing the reality of it. Antarctica is in shit conditions, wild fires keep getting more prominent, ice glaziers melting and other HORRIBLE realities of the consequences of not acknowledging the shit condition our planet is in. Do better.', 'Bernie considers climate crisis the most timely change to national security. He opposed the building of the Dakota Access Pipeline on the grounds that it will have a huge effect on the climate.\n___\nBernie Bot -Bernie_Bot_03', 'Woke capitalism is ESG.  It stands for environment, social, governance.  In short, they invest based on a firms stated values and beliefs (do they like puppies and rainbows), and not their financial performance\n\nESG funds are expensive and don’t outperform the SP index.  They also avoid fossil fuels because “climate change bad”\n\nIt’s wokeness injecting itself into the financial world', 'Climate change has its effects everywhere', '[The E-bike Is a Monstrosity](https://www.theatlantic.com/technology/archive/2022/08/ebike-electric-bikes-climate-change-exercise/671305/)\n\n\nMost of this article is just complaining about the aesthetics of e-bikes, but the highlight of this whole thing is when the author says\n\n\n&gt;\tWalking the streets of New York City, it now feels just as likely that you might get mowed down by an e-bike as a taxicab. \n\n\nAs if there is no difference between getting hit by a 65 lb e-bike and a 2 ton taxicab.', 'https://www.esquire.com/entertainment/tv/a23863674/george-rr-martin-game-of-thrones-politics-trump-climate-change/', 'Thats good. Meat should be more of a luxury to relieve farmers and to countrr the (dunno english) mass animal industry, which is the biggest problem of climate change', "Here in CA it costs less than half to go 300 mi on electricity. You will find cherry picked posts, but generally it's about half. \n\nThis means you have to justify the EV price with something else, gas alone will not make up the difference. Climate change, performance, time savings, etc. TOC is generally better with an EV of the same performance and luxury level.", 'our climate change conscious king 😍', 'I guess this would accelerate climate change and exterminate the human race even sooner. 🤔', 'Ah yes because the eleventh commandment was "thou shall not contribute to climate change"', "It's not economical. Uranium is a finite resource that's running out faster than oil. When it gets scarcer, the price will go up and it's already an expensive form of power. Because it's running out, it barely makes any difference worth mentioning to climate change but climate change already has most reactors in the EU shutdown, they can't be cooled properly because of drought.\n\nNuclear is not the answer lobbyists would have you believe.", 'Climate Change. It’s either seen as a hoax or an inevitable issue that ”we need to address soon”. I’m pretty sure the point of no return for our planet has passed by already, and we need to come up with solutions fast or else the EARTH IS GOING TO BECOME INHOSPITABLE.', 'Starts getting cooler in middle of October usually\n\nBut who knows, climate change and all now', 'The actual impact humans have on climate change.', "Why do you think that?\n\nI've never personally met a Christian who doesn't believe in climate change. I only know that exist because they pop up on documentaries 😂", 'The answer to climate change is space exploration, change my mind.', 'Is this the same Telegraph that on the front page today has an opinion piece stating that Boris will never be forgiven for Net Zero.\n\nHowever, against inflation we are seeing similar positions coming out in the US too. 17 US states have adopted anti ESG legislation for instance. Climate change is getting strong "don\'t look up" vibes.', 'One good source to read (The Xu paper I haven\'t read yet but will... really need to get to work ha) \n\nHowever, I\'d lose the institute for energy research piece in the future both for your own sake and trying to convince others. IER is a known fossil fuel and climate change denial lobby group initially founded by Koch but mostly funded by Exxon and the API (petriolium trade group). No matter if the piece is solid it will not be taken seriously by most environmentalist.  I\'ve found many of their pieces in the past have to "misinterpreted" the referenced data and sources or simply reference pieces from sister lobbying groups in a petroleum/climate denial misinformation self referential circle.', "Corporations are working hard to make common people believe climate change is a hoax because that's how they make the most money.\n\nFollow the money.", "c'est effectivement le choix de la décroissance plutôt que celui de la récession subie qui est le meilleur. mais ce n'est pas la direction qu'on prends.\n\nPar contre ce n'est pas le moyen qui est sûr en lui même, c'est la politique qu'on lui applique qui fait sa sûreté, or [cette sûreté ne pourra probablement plus être garantie dans un avenir proche en conséquence des choix politiques d'après le patron de l'ASN](https://www.lemonde.fr/economie/article/2022/01/19/la-poursuite-de-fonctionnement-des-reacteurs-nucleaires-d-edf-ne-doit-pas-etre-la-variable-d-ajustement-de-la-politique-energetique-francaise_6110102_3234.html)\n\nQuand à la stabilité c'est un point qui n'est pas garanti à l'avenir car on ne sait si le nucléaire sera adapté face au dérèglement climatique, comme mis en avant dans le papier publié dans energy policy: [Climate change, nuclear power, and the adaptation–mitigation dilemma](https://www.sciencedirect.com/science/article/abs/pii/S0301421510007329).\n\nIl y a d'autres choix qui pourraient (devraient?) être fait qui nous éviterait de construire plus de capacité production. Par exemple au lieu du non sens de la voiture électrique, on pourrait avoir des vélomobiles électriques. Si on remplacait la totalité du parc automobile par des vélomobiles électriques ça pourrait largement être alimenté par un fraction de l'éolien existant: https://www.lowtechmagazine.com/les-v%C3%A9lomobiles-electriques-aussi-rapides-et-confortables-que-les-automobiles-mais-80-fois-plus-efficaces.html\n\nEt il n'y a pas besoin de chercher bien loin pour trouver des postes d'économies similaires qui permettraient de réduire notre parc électrogène.\n\nOn peut facilement arguer qu'on consommera toujours tout ce qu'on produit et que face à l'urgence climatique il serait plus judicieux de réduire la capacité de production pour réduire la consommation énergétique.", 'It\'s fascinating to sit back and observe how impressionable folks can be in the absence of rigorous scrutiny of the body of science available in regards to "man-made" climate (cooling, warming, and now change) impacts.  Further, the elitists that set out to manipulate your opinions employ class politics to divide out society; pitting one group against another.  Unfortunately, the gullible fall for it every time. A simple review of catastrophe predictions during the 20th century by leading scientists of the day, will display a dizzying array of misguided, unrealized, poorly researched, and even poorer conclusions about the impending doom of the world.  Good luck making any sense of it all; you would have better luck with the local shaman or palm reader.\n\nIf you take the time to consider all the data, conclusions, and the most obvious contributors, you will quickly come to the conclusion it\'s the biggest hoax of the 21st century and the public is taking it hook, line, and sinker.\n\nIn spite of the skewed characterizations of white conservatives the only real thing you can conclude from the survey, is the fact they tend to be more educated, skeptical, and less prone to hysteria.  The balance of the conservative wing of the Republican party will exhibit the same attributes no matter their sex, ethnicity or age, hence the "conservative" moniker.  In other words, it\'s not a "white devil" thing...', 'Mm not necessarily but I have noticed an uptick in studies about climate change.', 'Then what problems do you perceive the government capable of solving? Since apparently climate change, economic depression, or mental health crises are not within their preview? \n\nIm seriously asking, since i dont think you are being realistic, what other entity is well suited to solve these issues facing literally every other nation on earth?\n\nIf you cannot answer why the goverment should not solve these problems, then you are a taxpaying m0ron. They’re collecting taxes either way they might as well solve problems or *govern*', 'Now it’s climate “death”. Did climate change just not motivate enough people anymore? \n\nWhen we’ve hit all the metrics that you said would instill “climate death” and nothing really happens are we going to get “climate mega death”?\n\nIs it getting hotter and the climate is shifting, yes. Has it been even slightly as bad as we’ve been told it was going to be for the last 30 years, lol no.', 'Bernie considers climate crisis the most valuable change to national security. He opposed the establishment of the Dakota Access Pipeline on the grounds that it will have a big result on the climate.\n___\nBernie Bot -Bernie_Bot_03', "Human-activity-related climate change accelerates and distorts these otherwise normal phenomena. The natural world isn't static and is punctuated by a handful of catastrophic events on geologic scales.", 'I fucking hate the "it\'s just a mask" justification for keeping mandates.\n\nSomeone could pass a law mandating that everybody clap their hands three times every morning at 10:10 in an effort to stop climate change. It might not stop climate change, but it won\'t make it worse and only takes a second or two, so it\'s not that onerous of a burden, right?', 'This isn’t just a group with an unpopular/easily proven wrong theory, like climate change is a hoax or trickle down economics is the best way to stimulate an economy. These people are sick. They have serious mental health problems to be able to justify their positions. Honestly, I’d say all Trump supports have a mental health issue, but Qanon especially.', "Oceanic climate + particles from coal and wood burning + no global climate change (yet) etc. would mean it wasn't as warm as it is today.", 'I can’t say with 100% certainty, but perhaps it’s because many American evangelicals lean politically right-wing/conservative, electric cars are probably seen as giving in to “the green climate change agenda,” and people nowadays mindlessly promote the brain dead memes and epithets of their political tribe.', "The following submission statement was provided by /u/radiomatrix:\n\n---\n\nSS: Pakistan's planning minister has said that the recent climate change induced floods have caused at least 10 billion dollars of damage. It was a monsoon on steroids. These floods have submerged a third of pakistan.\n\n---\n\n Please reply to OP's comment here: https://old.reddit.com/r/collapse/comments/x2l3i3/pakistan_floods_disaster_to_cost_more_than_10bn/imk1bsl/", "Climate changes naturally, but this climate change isn't natural so precedent doesn't apply", 'Climate change haha. Do you know where Canada rates on climate change? Let’s tax us more, raise the cost of everything, meanwhile we’re one of the lowest countries polluting. Nothing will come of it, unless other countries follow suit. Which they also haven’t. It’s a pipe dream.', 'What do you do about climate change, when the voters who keep you in office say "Let it burn"? \n\nThe problem is the people, which is why I am not optimistic about a solution.', 'As long as he says climate change bad. He can treat women how he wants.', 'lol...so you aren\'t smart enough to figure out why they have changed it.\n\nHurry up and go put on your mask snowflake.\n\nBTW...if you are so certain "global warming" is caused by humanity then why are you on Reddit? You accessed the site from a fossil fuel product produced in a fossil fuel consuming plant. If you think you\'re the cause then you shouldn\'t even be on here. Or are you just another "climate change" hypocrite who continues to bitch about a made up narrative designed to control you while you use your fossil fuel derivative products on a daily basis?\n\nlol...the cognitive dissonance in "climate change" snowflakes is next level', 'I want to make sure you understand that the ice sheet is always changing. It’s not a monolith it’s creating new ice and breaking off all the time from prehistory to today. The rate that it’s adding has slowed and the rate that it melts/breaks has increased due to climate change but it’s not all attributed to climate change.', 'Climate change isn’t caused by leaving trash on the ground.', "It's not even a plan, it's a natural progression for society. Notice how we don't live in tribal villages anymore. \n\nOf course climate change will be catastrophic, it already is, doesn't mean combatting it will require descent into dictatorship.", 'Wonder if it was far right white supremacy or climate change?', "I mean, the other option is to buy an EV from a company that hasn't been against Climate Change like Hyundai, Ford, and Tesla.  (GM was also lobbying against Climate Change laws...)", 'Even outside of climate change, the ice sheets are always in motion and changing during the winter it adds mass and in summer it breaks off and floats. Completely natural.', 'They only changed it to climate change because morons kept pointing at every cold day or snowstorm in their local area and saying "hur dur what global warming".  Climate change is caused by global warming, it\'s still happening, it\'s still caused by humanity.', 'A good way to frame climate change for people is that 1 in 1000 year storms become 1 in 100 years or even 1 in 50.', 'Then the doom of climate change is not real? I am only speculating that if the problems we face are catastrophic then the response will be as well. I am not concerned for me as I don’t think we are quite there yet. One world government is not a conspiracy but a well known plan.', 'Probably safe to just blame climate change', 'The biggest chance for human extinction in the next 50 years is nuclear war.  Nothing else is close.  Then maybe catastrophic bio-war.  If the world escapes those two, then the excellent writer John Michael Greer thinks for general economic and climate change reasons, there will be a slow decline for 200 to 300 years, followed by a Dark Ages.', "In all fairness Reading is very much known as a young person's festival. It typically falls on GCSE results day or thereabouts so you get a lot of young people. I was recently at Green Man, which for reasons mostly related to travel, is way more middle aged than most festivals, and it wasn't nearly as bad as this. I wouldn't be surprised if the reason Reading is so fucked up is because a lot of young people can't be bothered to travel home with their tents and shit, and those same young people are the people statistically most likely to be concerned about climate change and the environment.", 'Oh but you weren\'t the one directly attacking me, don\'t worry. \n\nIt\'s the guy who said that the issues with the schools is that they "push leftist agendas like LGBT" and teach stuff about climate change which they for some reason shouldn\'t, etc...\n\nI dismissed it because those are the classic conservative talking points, which I\'m tired of as they\'re always the same thing over and over again.\nI meant the actual issues of US schools (if there are), like , I don\'t know , the things they learn are too basic, or the organization is bad, all sorts of actual issues .\n\nIt\'s rather conservatives who wish what to believe. I say it because they all repeat the same arguments, as if someone or a group started to spread the arguments,  and people would just take it for word.\n\nLike I\'ve noticed that many people deny the fact that gender is a neurochemical phenomenon and not a social construct , something pushed by many conservatives figures like Matt Walsh . \nThat\'s just an example of how those people don\'t even seem to think about the argument and just parrot it around. I know not all conservatives are like that, but a decent amount sure are.', "Sounds very small fish you're frying there, given that the alternative will lead to a lot of deaths, wholesale reductions of the rights of the people in the UK, and failures across the board on important action on everything from climate change to trade deals.\n\nAnyone is electable if the people vote for them. Boris Johnson got elected for god's sake.", 'Evidence that god controls the climate and that humans aren’t wrecking it. Everyone knows about cyclical climate change over the billions of years the earth has been here. That’s not the climate change we are talking about and you know it.', 'Hahhaa bro I am not a cop. I can promise you that. Shit what do I gota do to prove it just tell me and il show you. I am also from Canada so that might be why.\n\nMy name is more towards the WEF and their agenda of a new world order and the shit that their doing to our countries without a vote from any of us people. Justice for all in my mind is to get the WEF the fuck out of politics and stop giving these people "fighting climate change " 1500 God damned  jets to fly in and out of conferences like Davos in this past may.\n\nI can see how it would look though from an outside perspective. It\'s also against the intelligence communities that have certain powers at play and use them against unknowing civilians. \n\nI gave over 76 charges and 32 convictions (here come the bad ass jokes when I\'m admitting  I was not a Good person) i stole from garages and sheds, cars, trucks, I was a dealer for many years. Raided three separate times over 6 years. Once was my parents home when I was young. I don\'t even know why I\'m telling you this. I have nothing to even prove to you, but I can tell you now I did 15 days for dboard in the hole, 23 and half a day 4 separate times and after each time I was was placed upstairs in placement with a cell mate and my canteen back only to get in shit and go right back down to solitary. All in all I did 6 months total of 23 and a half lockdown half upstairs with canteen and cellie  and half solitary.', '[Read the article in English.](https://www.deeperblue.com/activists-oppose-octopus-farms/) [](https://translate.google.com/translate?sl=auto&amp;tl=en&amp;u=https://www.deeperblue.com/activists-oppose-octopus-farms/)\n\n**Automated summary:**\n\n&gt; A coalition of 97 animal welfare organizations and scientists is opposing the establishment of farms meant to raise octopus and other cephalopods for food.\n\n&gt;“Government policy must focus on shifting to a global, plant-based food system in an attempt to tackle the dire climate change emergency we are currently facing worldwide.\n\n&gt;Banning these types of farms now, will allow countries to develop in complete alignment with the 2030 SDG commitments moving forward.”\n\n&gt;He first got the diving bug while in High School in Cairo, Egypt, where he earned his PADI Open Water Diver certification in the Red Sea off the Sinai Peninsula.\n\n&gt;Since then, John has dived in a volcanic lake in Guatemala, among white-tipped sharks off the Pacific Coast of Costa Rica, and other places including a pool in Las Vegas helping to break the world record for the largest underwater press conference.\n\n&gt;\n\n\n\n**More:**\n\n* [Bay Bridge March Marks World Day for the End of Speciesism | Indybay](/r/VeganLobby/comments/x0div9/bay_bridge_march_marks_world_day_for_the_end_of/)\n\n* [Farm leaders vow to overcome vegan dairy shutdown | Farmers Weekly](/r/VeganLobby/comments/wxtrd3/farm_leaders_vow_to_overcome_vegan_dairy_shutdown/)\n\n* [World Day for the End of Speciesism: PETA ‘Animals’ Feast on ‘Human Flesh’ | PETA UK](/r/VeganLobby/comments/wz6gtl/world_day_for_the_end_of_speciesism_peta_animals/)', 'based on Ethiopias largest model: 460 geothermal plants can power the entire US. However after it’s built, no one would ‘profit’ because it’s a one and done deal. 100 trillion in stranded assets if climate change is addressed. Monetary market system is obsolete', "The people angry that their taxes are supposedly paying to forgive some student loans for other people say that we should spend our taxes fixing their state which is underwater due to climate change which they say isn't real.", 'Technology is improving cars and making them more resilient to rust. Ontario and Quebec winters are also getting much less intense due to climate change. The average car in Canada lasts 13 years now.', 'There are a growing number of us in “that generation” that are taking climate change very seriously and are mobilizing in non-trivial ways.\n\nWhat you fail to see here by focusing on one generation is that the majority of humans across any cohort are not productive members of society at all. They are net-takers and are happy to leave a message behind.\n\nThere a very small group of people (proportionally) that keep the world spinning. From custodian/garbage man to engineers/physicians - they are willing to put in the hard/dirty work to keep society going.\n\nWithout that tiny minority, this festival wouldn’t be possible.\n\nAnd yeah, that hypocrisy is not limited to the younger generations. I’d bet boomers (or whoever else) are just as bad if you were to measure.', "San Francisco for really mild weather. No extreme top or bottoms. Wild fires and earthquakes, though.\n\nDuluth, MN. Will be pretty immune to climate change. Just snows a lot. Maybe not mild in the sense but if you are worried about the future of weather, it'll put you in a good spot.\n\nI just moved to Cincinnati this past week. I chose it due to it's more mild weather. There are a good amount of thunderstorms and rain, but it's sporadic and passes quickly. Tornados exist but are a rare thing. It barely snows or gets below freezing - less than half the national average snowfall. It doesn't get as hot or humid as Texas where I was coming from - only a handful of days this past year were above 90. Weatherspark gives it some hot/humid day rankings, but Austin, where I was coming from, would be humid every morning and have a level of some uncomfortable humidity most of the summer. You couldn't escape it.", 'I work 2 jobs right now, 1 is in sustainable agriculture. The other is supporting adults with disabilities. The one job I am live on site for, the other I commute 32 km to. Which job are you relocating? Half the staff at the group home commute there because the the town is less than 3000 pop. You want to move the disabled people out of their small town where Everyone knows them and to a big city, just so you can have your dogma?  Or are we somehow moving the farm closer to the workers, longest commute now for the farm is 20mins by car. Certainly aren’t running trains to my farm or to the tiny village that I work in. It’s almost like there are a multitude of use cases for personal vehicles. Am I bringing hay in on a train in the fall? Or renting a truck that I need to somehow pick up without a personal vehicle. How are those disabled people getting to anything in the area, there Drs are in a different town, hell their groceries are in a different town. We have staff at the group home commuting 1 hr from the north to work, moving the group home eliminates them as staff as the town with the most staff is 30 mins south of where we are currently. Those staff that live 1hr north now don’t have a job or a way to get one cause they live in the wilderness where there is no public transit  and there obviously never will be. I literally have no idea what that 1st sentence is referring to, since I work in ag, all climate change effects me.', "You can't shame or correct these conservatives (including their supporters) using gotcha moments and hypocrisy. Every single issue - you can catch the right making disingenuous arguments. They don't care. Coherence and consistency of rhetoric and action is not their goal. Power and hegemony are their goals, and hypocrisy is just one strategy for accomplishing it. In fact, it's become a sport for them: rack up the most points, and any opposition focused on your words will never be able to hold you accountable for your actions. Kayfabe and theatrics.\n\nIn effect what you have is a wealthy elite that have weaponized social media in order to outsource their own PR/propaganda/public defense to gullible and disaffected individuals. The fascist engine works by transmuting populist anger and stereotypes into political power. If you look at what right wingers say vs what they ignore, only one viable conclusion emerges: The rhetoric is a smokescreen. They only care about power, preservation of power, and withholding of power from people they don't like. And they'll say anything to further desires lurking in the muck of their souls. All the while contributing to a festering crab-bucket mentality that drives intra-class conflict. \n\nDon't look for logical or semantic consistency across fascists' statements and beliefs: the real commonalities are who they affect in that moment: protecting in-groups, or hurting anyone that threatens that. And by the way some behave, existence alone can seem a threat.\n\nEdit: Rather than just bleating about a problem, I could also suggest a solution. Don't give them the attention they need. Whenever you hear a fascist hypocrite serving up their steaming, crappy hot takes, just tell them to shut up. Ignore them, mock them if you want, but don't engage with logic. And more important than wasting attention on right-wing fascists - **get anyone and everyone you can to vote**.  When you talk politics, don't talk about hypocrites - talk about real issues affecting real people, and how progressive policies like taxation of the wealthy, mitigation of climate change issues, safe and fair labor laws, and access to healthcare and education can improve the life of the average person.", 'I’m not saying that it is. \n\nClimate change is displacing people, either we make the climate a place where ice sheets are thinning less quickly, or we need to displace. I hope that we can do that while we still can. Many can’t afford to be displaced.  And I don’t know many who want to.', " these people only have to scramble around filling shifts, because if the worker's were treated comfortably, they would have more time to resist and build an alternative system. thus they are only really needed by the parasitic rich at top to run as interference to the workers, and aren't really needed by the worker's themselves. further as you stated, they don't really come up with new ideas or make things better in anyway. further, there's been ton's of historical examples of the workers themselves coming up with new ideas, or implementing things better, so the rich at the top are only there because of the weight of their ancestor's rigging things in their favor. the collapse will come to some places faster than others, primarily the red states in the path of climate change. while you might find new York rebuilding, no rebuilding's going to really take place in Louisiana. you can already see this in the data, both republicans and those living in red states have a lower quality of life and life expectancy, and it's only going to get worse from here. while MIT has it at 2040 for societal collapse, i think some places will collapse earlier than that, maybe 2030 for the poorest red states. further, this is just going to further inflame fascism", 'Look like he didn\'t bother learning the difference between mood and emotion. Mood = long term, emotion = transient.\n\nLike weather, day by day the weather can change, but for a given region it will have a general climate.\nNobody says today\'s weather is "mild temperate oceanic climate" because that makes no sense.', 'Climate change, Russia, January 6 and Genders', '"Yeah, you see, we had a normal summer, climate change is a myth, what? So what we have 30 degrees in december, we had a normal summer, climate change is a myth" You', 'Due to a number of factors, including climate change, I think more people who actually would quite like children, will also be choosing not to have them.', 'The globalization we chose in the 80s has played out pretty much exactly as those opposed predicted so you’re right, the 90s was a peak. Reagan-Thatcher-Mulroney-Xiaopong changed the world. \n\nClimate change is also going to be bad, already causing long droughts. We’re in pretty deep denial but it’s coming.', 'Leo on dating sites like: \n\nDislikes: fully developed pre-frontal cortexes, climate change.', '&gt;Here is another scientific study \n\nlol...this statement literally means nothing anymore. "Science" has become nothing more than yet another controlled and manipulated "source" for furthering an agenda.\n\nCase in point...Convid aka ....the scamdemic. 2 years later we have all the "science based experts" completely reversing their previous bullshit "science".\n\nSure there is real science in the world but anything used as "science" in the mainstream pushing a specific narrative is 100% bullshit.\n\nLike your post about "climate change"\n\nDo some research. \n\nIt started out as "Man-made global warming"\n\nThen they changed the moniker to "global warming\'\n\nNow it\'s called "climate change".\n\nI wonder if you are smart enough to figure out why they have changed it over the years?', 'I wonder how many of these people froth at the mouth about waste and climate change', 'My brother thinks the NASA web page on climate change is fake news', 'As long as a little drone flew around behind everyone who pushes this notion and would instantly shoot them for eating meat. Same thing with the climate change awareness jet set club.', "humlog bahut sahi time pe paida hue kyunki it's the information age kuch bhi sikhna is so easy. kabhi kabhi main soch bhi nahi sakta hoon ki pehle ke coders ko bug fix karne ke liye kitne books tatolne padte. internet has made everything faster, from having a conversation with someone to ordering groceries.\n\nclimate change se bahut dikkat hoga aane wale samay mein toh aane wale generation ko bahut dikkat hoga.\n\n70s 80s ke bollywood movies jo ki gaun mein shoot hote the usme door door tak factories aur sadak nahi dikhte the. nadi pahad yahi sab dikhta the. abhi toh har jagah concrete jungle banne lage hai.", 'Ok well you keep living in your world of bubble wrap protection. Why bring climate change into this? Makes zero sense. Canada was one of the highest vaccinations rates in the world, we also had some of the longest mandates. People can isolate all they want, just don’t expect to paid for it.', 'No, they are saying that evidence is inconclusive.  It\'s easy to do that when you get to cherry pick the data.  This is fucking Rand we are talking about.  They likely have 20 year old similar *studies* on climate change data being "inconclusive" based on the *evidence.*', 'nah nice try not reading literal thesis on why climate change is horrible, soecially when that thesis is heavily bias.', 'That and climate change, which changes the jet streams and creating hander more erratic winters.', "SS: Pakistan's planning minister has said that the recent climate change induced floods have caused at least 10 billion dollars of damage. It was a monsoon on steroids. These floods have submerged a third of pakistan.", 'Lol, nobody has been “living like this” for a while re: COVID. It’s been a free for all for a while. \n\nIts pathetic that society expects people to NOT isolate during any transmissible illness. Pathetic that we will baby the economy by unnecessarily forcing people to commute but not touch climate change with a 10 foot pole. \n\nMost office workers can do most, if not all their work from home so there is no point in “switching back” to the normalized notion that going into work, school etc. while sick is a good idea. \n\nFuck going back, I’m fine with people isolating if it means less chance of contracting their illness. Also some humans will always live with weakened immune systems and isolation is a great barrier to keeping them safe. They’re not always visible so again, preventative measures works best. \n\nIf you want to go to work sick, fine I guess, just stay in your cubicle/forklift/space and definitely expect me to think you’re a piece of trash for risking my health.', 'Because the "infrastructure bill" was mostly pork, for stuff like mileage taxes, and various "climate change" crap that would fuel even more inflation.', 'That still happens. See any coverage of climate change.', "That's so strange...... It looks like all the 'money' is pushing the climate change narrative.", 'There’s nothing interesting about humans leaving their fucking trash everywhere and not cleaning up after theMselves. \n\nWe deserve climate change if this is how we willing treat our fucking planet.', 'Its not a mistake. Theyre wrecking everything under the guise of "climate change".', 'Well you could think about your comment for a second before posting. See I was commenting on an opinion, climate change evidence is scientific, so no opinion needed there. If you can show me a consensus of 97% of set designers/costume designers/movie makers in general that agree the production quality of this show is garbage, then you may have a point', 'Okay I’m not sure that’s my point. It sounded like you were saying “The summer was hot once and the media were stoking fears about climate change” and my point was “No, they were using the miserable hot summer to illustrate the problem, but the proof of climate change is actually a different story.”\n\nA well-documented and accurate story. But nobody was using one hot summer as the only reason to show that climate change is real. Last summer was just an example.', "&gt;  It's definitely climate change.\n\nFuckin knew it.", 'None of those things address climate change. What specially makes you think we’re going to outlast the massive changes that are coming? Where’s there food and water gonna come from?', 'There are ecological models predicting that climate change, pollution, and overfishing ~~wool~~ will transition the oceans to the point that jellyfish become the dominant form of life. \n\nI like that you can also view it as a metaphor.', 'That is my point. People incorrectly use moments of extreme weather as evidence of climate change, which is just confirmation bias. Actual climate change would be a global phenomenon recorded over years.', 'they are the factory for the entire world and have a population of over 1.5 billion people, ofc they’re gonna be one of the largest polluters but at least they’re actively trying to move to sustainable energy unlike America. besides, climate change is an issue that can only be tackled thru global cooperation, china alone can’t fix this.', '“It gets better” they say, climate change still unmanaged. Even the most beautiful castle is fucked if it’s built on a swamp.', 'Dan wore long pants to a concert in Miami in August and blames climate change that he’s hot.', 'I’m sure everyone there made a Twitter post about how they’re worried about climate change.', "So far I have these books on my reading list:\n\n* Overshoot: The Ecological Basis of Revolutionary Change\n* The Limits to Growth: The 30-Year Update\n* Collapse of Complex Societies (I don't know if I'm really going to read this one, though)\n* The Uninhabitable Earth: Life After Warming\n* Our Final Warning: Six Degrees of Climate Emergency\n\nI'd appreciate hearing about your favorites.", "Curious that in 2022 recommendation algorithms can't differentiate between positive and negative connotations. It's like getting recommendations for a sauna after you check stuff on climate change", "I don't know many younger people that even want these things anymore outside of owning a car and I don't think I even know anyone without one of those. How many women do you know that actually want to be a housewife or young couples that want to move out of the city to the suburbs same goes for having kids most don't want to spend the money or have their free time taken away or just don't like the way the world is heading with climate change etc..", "Seriously this. The carbon tax was originally proposed as a *Conservative policy* and was coopted by the Liberals, it's a way to use market forces rather than government regulations to push companies towards using less carbon. It's pretty much the most milquetoast climate plan possible that has a chance of accomplishing even poor results that are too little too late. But now that the Liberals put it in place suddenly it's the just the worst response to climate change possible and the Conservatives are scrambling to find an even less effective option.", 'Lmao Fraser Insitute hack and former  conservative politician. Can safely ignore.\n\nIt would be highly irresponsible for pension funds whose investment horizon spans decades to ignore real tangible risks from climate change in their investment analysis and decision making.', 'This is exactly how I try explaining to my trump neighbors who view the world through the lens of their farms being the whole world. If they want a say they have to bring their concerns and policies to the table with sane leaders who act on good faith and doing the right thing. Part of doing the right thing is not lying to their constituents over and over again and feeding them lies bigger than they can understand. They can’t understand anything other than that the sun rises in the east and sets in the west. They don’t understand that due to climate change their conservative ideas kill their whole entire lifestyle. They have been breeding ignorance for years and the brain drain is showing itself right now. I moved back home to rural MN after hopping around the country to experience life outside of the rural perspective. I like it better here and I understand where the rural ignorance comes from; it comes from the root of everything we all want, prosperity and peace. Unfortunately they have been voting in the bogeyman for so long they themselves have become the bogeymen.', "If you haven't noticed the reputational damage done to science the last few years, by trained scientists prioritizing their (primarily political) biases then I suggest you haven't been paying attention.\n\nSo let's refresh...my OC is that these particular individuals are choosing to act as political activists instead of objective scientists. They've admitted as much. So what exactly are you disagreeing with?\n\n&amp;#x200B;\n\n  \nEdit: to synapsebondfire, my favorite kind of interweb debate-lord who when intellectually pushed just the tiniest bit, panics and resorts to the tried and true strategy of replying to, and then immediately blocking, the person they disagree with. If I had an award to give..you earned it little buddy. Now, to keep the hierarchical flow:\n\n&amp;#x200B;\n\n&gt;Says the guy with nothing but vague accusations. I assume you’re talking about climate scientists. And if you’re denying climate change I just assume you’re an idiot, which is a very safe assumption at this point. Also done arguing with someone screeching ‘omg people are political at times and that’s terrible’ which inherently a political statement in and of itself. You have no argument just empty vague accusations that are meaningless to begin with\n\n  \nLet me help ya out here...I was referencing primarily the covid shenanigans of most 'science based' public institutions and the wokism infiltrating higher ed but absolutely some climate scientists (like the ones in the article) haven't done themselves any favors either.", 'Kind of. We often plan for what we think the future is going to be. Like: “the consequences of climate change are going to suck, so we should reduce our dependency on fossil fuels and switch to electric cars” what happens tomorrow influences how we behave today.', "I am literally eating oatmeal with flex and sunflower seeds, nuts and multivitamin drink.\n\nI get this question right when I eat the healthiest meal of all time and it's also vegan so climate change will be helped A LOT too.", 'So, jumping to conclusions like this are a bad idea.\n\nWhat happens if next year we get a mild year with averages of temperature, rainfall and everything else? Do we assume climate change is over?', 'Thanks for the heads up on this! I was going to order the clear cover, but now will skip it. This planner is going to live in a drawer at work, so it’ll be fine to expose it to the world. Dog knows real polar bears are in greater danger from climate change induced habitat loss, neh?', "The air is warming. The oceans are also warming.\n\nWarmer air holds more water. Warmer oceans evaporate more water and generate more humid air. \n\n&gt;\tFor one to form, there needs to be warm ocean water and moist, humid air in the region. When humid air is flowing upward at a zone of low pressure over warm ocean water, the water is released from the air, as creating the clouds of the storm.\n\n&gt;\tAs it rises, the air in a hurricane rotates. Air drawn into the center of a hurricane curves to the right in the Northern Hemisphere and toward the left in the Southern Hemisphere due to the Coriolis effect — a phenomenon in which winds curve because of the Earth's rotation. At lower latitudes, where there is no Coriolis effect, hurricanes cannot form within 300 miles (500 kilometers) of the equator.\n\n&gt;\tStorms grow if there is a continuous supply of energy from warm ocean water and warm, moist air. Tropical storms can grow into hurricanes, and hurricanes can grow into stronger hurricanes. However, only a small number of storms grow into tropical storms. Even fewer become hurricanes.\n\n&gt;\tStorms weaken when they move over areas with cooler ocean water. There isn’t nearly as much energy in the water to fuel the storm, nor is there as much humidity in the air. Hurricanes also weaken when they travel over land.\n\nTaken from here: https://scied.ucar.edu/learning-zone/storms/how-hurricanes-form\n\n&gt;\tLarge sections of the global oceans have warmed 1-3°F during the last century, with some buoy locations having a more dramatic rise recently. Of the Gulf of Mexico buoys analyzed, the average water temperature from August through October has risen 1-2°F in the last 40 years. The buoys analyzed in the Atlantic have warmed even more during that three-month period — 2-4°F in the last 30 years. While at single buoys the water may have warmed faster or slower than other locations, globally, there is a clear trend towards hotter sea surface temperatures.\n\n&gt;\tWarmer water fuels hurricanes, and not surprisingly, the average number of major hurricanes — those Category 3 or higher — has increased in the Atlantic basin since the 1970s. Globally, climate models project that there will likely be an increase in the wind speed and rainfall intensity of the strongest hurricanes (aka tropical cyclones), even though their frequency may remain unchanged or even decrease by the end of this century. However, the extent of these effects from climate change will vary by region.\n\n&gt;\tEven with no increase in storm strength, the damage from storm surge is already worse because warming waters are also contributing to sea level rise. Seas are projected to rise up to at least another 3 feet by the end of the century, only adding additional water to any tropical system’s dangerous storm surge.\n\nTaken from here: https://www.climatecentral.org/climate-matters/us-coastal-water-temperature-trends\n\nTLDR: Atlantic, where hurricanes that end up in the Gulf of Mexico form, is warming. The Gulf is also warming. Warmer Atlantic means more energy to feed the air cycle of a nascent storm system and help it develop into a hurricane. The warmer Gulf will help ensure less energy is lost as the system moves from its forming grounds. Rising sea levels (they are already rising) will also make storm surges worse. \n\nThe best we can hope for is an increased frequency of storms that we can just about manage. The worst case is the same frequency we’ve seen in recent years, which is up on historic averages, but with more of the mega Cat 5 systems actually making landfall.\n\nI wouldn’t buy property anyway on the Gulf cost. NOLA is going to be destroyed again within my life time and it won’t be viable to rebuild. Miami will be underwater within my lifetime. Those refineries on the LA/TX border? They’re going to get a pounding one day (again) and it will be a major disaster.", 'Robert Patrick doesn’t believe in climate change and would regularly walk around set shouting “MAGA MAGA MAGA.” He is not an intelligent dude.', 'Yeah, sort of like how all the other climate change forecasts have turned out totally accurate and not insanely conservative compared to reality. The planet is on fire.', "Climate change is real and it's caused by man-made emissions of CO2. Global mean surface temperatures seem destined to increase by 3.5-5.5°C by 2100 as a result.\n\nHowever, none of that means Victoria Gonzales will have an average maximum temperature of 30°C on August 30th. Even by the year 2100, with the expected warming the average maximum temperature will be closer to 20°C than it is to 30°C, probably in the vicinity of 22-23°C.\n\nEdit: 3.5-5.5°C. I'm all thumbs.", 'I like the guardian. \n\nDon’t really give a fuck about climate change though so can’t comment', 'Bernie considers climate crisis the most valuable change to national security. He opposed the manufacturing of the Dakota Access Pipeline on the grounds that it will have a huge result on the climate.\n___\nBernie Bot -Bernie_Bot_03', '&gt;This week, the Lever, ProPublica and the New York Times  \n discovered the largest known political advocacy donation in American   \nhistory. We exposed a reclusive billionaire’s secret transfer of $1.6bn   \nto a political group controlled by the Republican operative Leonard Leo,  \n who spearheaded the construction of a conservative supreme court   \nsupermajority to end abortion, block government regulations, stymie the fight against climate change and limit voting rights.', 'That was literally the PR policy of the oil companies when they realized climate change was going to be an issue.  Treat it as “just a theory”, demand “equal time”, deflect, deflect, deflect.', 'Lul ain’t nobody scared of no climate change climate change is natural', '“Climate change doesn\'t exist" mfs be like.', 'Its probably influence from sci-fi films, combined with the current state of the world and outlook of the future especially in regards to climate change.', "So what's their solution? Not find an alternative, but to pretend the oil is infinite and that climate change doesn't exist because we're all off to an eternal orgasmic afterlife anyway as long as we obey a randomly homophobic sky patriarch.", "I don't see how they can even pretend to care about the environment or climate change anymore, especially if they elect PP. You can't campaign on getting rid of the federal carbon tax (an economically illiterate position), getting rid of environmental protection legislation (bill C-48 and C-69), expanding pipelines, and expect people to take you seriously on environmental issues.\n\nConservatives don't want to conserve the environment. Everyone already knows this. Rick Perkins and many other Conservatives fight against fishing moratoriums. If a green conservative was in the leadership race, they'd be called a Liberal instantly (if they even get approved).", '&gt;Giving the American people the right to vote on Abortion is not extreme. What\'s extreme is taking away that vote by letting SCOTUS decide.\n\nShould we also give Americans the right to vote on whether blacks count fully or just for 3/5?\n\nOr if we should allow "Whites only" signs on water fountains again?\n\n&gt;As a country we\'ve always believed that there are things not listed in the constitution and that states have a right and obligation to handle those things the way they see fit. There\'s no right to Abortion in the U.S. constitution.\n\nAnd literally every time we\'ve done that it\'s been used to oppress people until the federal government had to step in.\n\nBut continue arguing in favor of the state\'s ability to *force* you to give birth, because that\'s not fascist at all.\n\n&gt;That being said, the only people concerned that Roe was overturned is the far left. Right now Americans can\'t afford gas, food, and necessities (or those necessities are gone). That\'s what they are focusing on and the left is doing nothing to fix it. We are in a recession and all the left can do is spend money on "climate change" when they should be trying to get us out of this recession.\n\nMan, it must be nice to be able to just completely ignore the half of the Senate *you control* refusing to agree on so much as a lunch order.', 'Yep…bloody flat-earthers, climate change denying plague rats, won’t understand simple geography', 'I bet these same people are highly concerned about climate change.', 'I believe in climate change. I just know even if all the average Joe’s in the world did everything they were supposed to, the rich and powerful would still burn everything down on their own, so I refuse to be guilt tripped into eating bugs so celebrities can take their yachts to Malaysia.', "/unchurch\nThe fibrin scar tissue that occurs as a result of damage to the myocardium allows for/causes clots to form. These clots interfere with blood movement, sometimes resulting in pulmonary embolisms.\nThe injured heart tissue itself can also lead to complications of atrial fibrillation.\nNow, adding intense exercise exacerbates these issues and those who were unfortunate to have their body end up with these complications have a higher chance of mortality.\n\n/Rechurch\nIt's definitely climate change.", 'I agree with this. Just saying it is just gonna scare away people on the fence. We’re in the realm of being able to focus on things like climate change, cancer, heart disease etc but we’re focusing on guns.', 'if those gaudy islands in the middle of climate change is how they are spending their money, I would not put a lot of faith in that', "It's a legitimate anxiety to have.. Sorry to say. It's bad. But, society won't collapse until around 2070 (if no measures are taken). And even then, people won't start dying by the millions from it for another few decades. Once again, this is if nothing is done. Millenials and Gen Z will be coming into political and government roles soon, instead of old conservative whit dudes. That gives me hope. I've had to do several projects on climate change.", 'Who needs fish? Who needs forests? Who needs land animals? Who needs insects? It is all a lie... we are to rule this planet..it is ours...we can do with it, whatever we want..climate change? Leftist propaganda...God wills it ...Deus Vult...\n\n/kappa', "No, you're oversimplifying.\n\nClimate change can cause drought, ocean acidification, polar ice cap melting in 30, 50 or 100 years. Or shorter. Or longer.  The clathrate gun permafrost might happen sooner. Or later. Or happen and be overstated.", "...I know what climate change is. it's a great side effect of veganism but *it is not what veganism is about*", "Yes, climate change will be the end of many species on earth if it isn't reversed. Say goodbye to penguins and polar bears if all the ice melts. Deserts in Australia and Africa are spreading. African wilife is struggling to find fresh water. Ocean acidification is only getting worse. Coral reefs are dying. Earth's wildlife is in serious trouble and will be pushed to the limit in the next few decades. \n\nAbolishing animal agriculture hopefully will happen in the next few decades. Or at least drastically reduced. I think it's inevitable at this point. This means we won't be using any animal products as we'll need all the resources we can get to feed humans when droughts and heatwaves are more common.", 'The heat now is nothing compared to what it will be in 30 years lol, there have been heat waves all across the world and climate change hasn\'t even "started" yet. You are right though, humans are adaptable. In this case though, humans in the Southwest will likely have to adapt the way people in Dubai live now. Only coming out at night, never during the day, wearing cloth over their entire body head to toe, etc. Scary thing is all the rivers will dry up, so they will only survive on their groundwater, for however long that lasts... Not sustainable at all in the long term of humanity.', 'Memo from pedo pete saids all the increase in deaths are because of the Jan 6 th insurgants,putin, antivaxer,conservatives, white heterosexual men and climate change.  Anyone who differs should be banned,censored and be labeled a fascist nazi.', 'Climate change', 'Climate change', '&gt;Pennsylvania\'s GOP gubernatorial nominee, Doug Mastriano, deleted 14 videos from his Facebook page in which he touted far-right positions, including referring to climate change as “pop science."\n\nDoes anyone have these videos?', 'Worldwide it’s a small minority. In the USA, it’s more than 1/4 of the population.\n\nhttps://climatecommunication.yale.edu/publications/climate-change-in-the-american-mind-april-2022/toc/3/', 'Would it make you feel better that Justin Trudeau says climate change is the reason for Everyone anxiety.', 'Like climate change?', "climate change weren't we overdue for an ice age", 'Nobody is making bugs for human consumption you chuds, and even if they were, it’s the least we can do for the climate.\n\nNobody is saying forever, just a little big longer.\n\nYou might be done with climate change, but climate change isn’t done with you.', 'Not a summer in North Central Texas. \n\nEven with climate change cooking the books, these are early fall numbers.', "I am just tired of people arguing over covid / nuclear or not / whether climate change is real or not.\n\nFuck this. I'll come back once people have some sort of common sense in them.", 'The EU will solve it by adding further restrictions on farmers in the name of fighting climate change.', 'I’m not sure how productive this is because you are missing my entire argument. The point is that a small child thinks it is impossible to read before they are taught how to do so, and they feel the way that you feel about climate change. The overly pessimistic fear that there’s no way it can be accomplished can be overcome. \n\nThe same holds true with respect to facing the dangers that exist in the world today. Every single generation has had its own set of hardships and reasons why it thought it’s kids were in danger. That doesn’t mean they shouldn’t have kids.￼ it does mean we should teach them to face those challenges.￼￼\n\nAnd just to take the bait on your mass shooting argument, what that means to me is that we have to teach our children that there are ways to make changes in our society. If you don’t like the way things are going today, get out there and vote to make a difference. Get out there and talk to people to make a difference. Get out there and show people what you want the world to be like, and build a coalition around it. That is how our society works.￼ Your view is evidently different. \n\nI don’t want to be in your camp, where the existence of enormous problems such as climate change and mass shootings means we should allow humanity to go extinct.￼ You may think I’m crazy, irrational, overly pessimistic, naïve, etc. But there is simply nothing that you have said that leads me to believe that the best solution to our problems is the extinction of mankind.￼', 'Hmmm I think we should write to our politicians about climate change. Maybe they can help', 'First i was like "aww, sweet. They\'re still together and they\'ve given up on driving a car to battle climate change." Then i realized what sub this is in.', "I often try to include fundamental changes in my world's history. All in all makes it more realistic as change is the essence of reality. I even have maps from different stages in my worldbuilding process that are all valid because the world changed due to tectonics, climate change etc.", 'Yes. They are all possibilities that should be explored. We would likely not hear about the geologic record having multiple periods of climate change indistinguishable from industrialized climate change unless it also had a spin that was acceptable for publishing. That’s a problem that should be addressed because if we have hypothetically taken a wrong turn it will take longer to correct when bias has enabled it to continue far beyond the scientific method. Facts are facts and they should be able to publish at face value and shouldn’t require an acceptable narrative to pass publishing. I speak partially of this article because it appears to me to be the case here, as I feel they’ve reached around the obvious and controversial to be accepted for publishing but even if that wasn’t the case here, it is a prevailing issue.', 'I hear he’s also a climate change expert', 'Will they still be around with all the climate change?', 'Wait until they hear about [NFTs that help fight climate change](https://co2compound.klimadao.finance/)', "People aren't saying that the MBTA should pay for itself with the exception of only a few minor voices in the room. People are often accused of arguing that point when really they are just asking for the MBTA to be partially funded by fare revenue so utilization of a capacity constrained public service isn't fully free to the end user. Which is the change that advocates are trying to change by making the MBTA free. \n\nI can see the counter point as well which is that the poor largely get hit by that funding source but that is true of the gas tax as well which many pro-public transportation advocates actually want to increase for the climate arguments.", "it is my belief ghosts are part of another dimension. I believe ghosts will still be here because the PHYSICAL earth will NOT be destroyed, humans just won't be able to survive the climate change.", 'Best to not say anything. Don\'t want to offend any of the poor oppressed children. Just do your job, get shit on for being in a lower social class and make sure our future leaders know their better than you.\n\n\nEven funnier is that these are the kids who march out of class to bring attention to Climate Change and pollution, while Ignore their contribution to it.\n\n\nAND EVEN FUNNIER, the response in this thread. From "blame the parents" to "Do your job janny". So much wokeness and equity....', 'Ok so to answer your question hypothetically: nothing. If the earth is completely destroyed by climate change and say all humans go extinct then nothing. Ghosts would just be living in a now very polluted and empty planet, they cant die so yeah. The end.', 'Its possible Ordered new corepads but since some other people report this issue I dont think thats the problem.\n\nSo what climate are you living in. You dont feel the initial friction change at all? when I first got it it was super speedy, and now its just mud pad sometimes.', "People could very easily correlate if they thought for just 5 seconds but they just don't want to when they don't like what science says. My main point was that people see scientists as the people bring bad news like things about covid and climate change and start being anti science while using the technology developed based on science. Kind of like someone on ACA claiming that Obama care is bad or someone getting their loan forgiven claiming that loan forgiveness is bad when others get it. Point is not that people don't know but that people loving something that helps them and hating it when it helps other or asks them to do something they don't want to do. Point is to point out the weird mental gymnastics they do.", 'I\'m a man and I listened to Jordan Peterson, and I can say what he\'s trying to teach me is none of that. He is not doing that all in any context. He\'s using those phrases/words, but under the guise of being racist, sexist, and overall bigoted. Being responsible and cleaning my room are basic things I already learned from my mom. Jordan Peterson\'s appeal is that he tries to rationalize being a "man" with extremely outdated and harmful views, in an attempt to uphold the power structure White men used to have. \n\n  \nAlso he straight up doesn\'t know anything about [climate change](https://www.theguardian.com/environment/2022/jan/27/word-salad-of-nonsense-scientists-denounce-jordan-petersons-comments-on-climate-models)(but still thinks of himself as an expert on the topic). His comments on Black Lives Matter were horrendous(and shows how completely limited is viewpoint is), his PhD research has nothing to do with like 90% of the topics he talks about and has no expertise in.  \n\n\nIf I wanted to be someone who does no research and talks nonsense, sure Jordan Peterson is a great listen. If I wanted to learn to clean my room, there are people who I can talk who aren\'t a bigot like Jordan Peterson.', "veganism isn't about climate change.", 'We wouldn’t have to worry about climate change.', "with the mountain of data supporting the cataclysmic danger man-made climate change poses, i'd say there's much more understating the danger in the media than overstating.\n\nthat said, it's not hopeless, there's been a lot of positive change in the last couple of years, so despite megacorporations kicking and dragging their feet the whole way, things are looking up. i don't know if you need to hear it's not hopeless to take the threat seriously, but if so, there it is", 'I have a plan to fix climate change. What we’ll do is give more subsidies to oil and gas corporations, while mandating every small town has a never ending burning tire pile.', 'Terrorism is performing acts of aggression in an attempt to cause fear and thus force someone to change what people are doing. The problem is that it doesn\'t work, the people that you kill will be replaced and the rest of the people will just dislike your ideas and beliefs more because the vast majority of people doesn\'t like violence. And now imagine that in a country where the government can spin that bombing into an already established narrative of "us versus the west". Yeah, you are just making Russians agree with Putin\'s rhetoric more...\n\n"Comparing someone dropping a candy wrapper on the ground to an invading army committing war crimes across your nation is pretty ridiculous TBH and I’m not even going to entertain that."\n\nComparing the damage done to the climate worldwide by large scale polluters to someone dropping a candy wrapper on the ground is pretty ridiculous TBH and I\'m not even going to entertain that...\n\nBtw, did you consider that maybe there is a person in those photos that may have done the bombing in question, but he is just having a day off to relax after a hard months work of secret freedom fighting/terrorism? Don\'t judge people for enjoying their free time, you go crazy if you just spend every minute of every day thinking about how to kill Russians. Or maybe you are already crazy...', 'climate change, i mean thats not just in New York but you should still be afraid', 'Or that perhaps the effect we are having on the planet is at the scale of previous natural occurrences of climate change.', "We need more climate change so it's consistently warm enough where I live to ride a bike everywhere.", 'NDP and Liberals are showing how little they actually care about acting sufficiently on climate change. The Greens are also recovering from the Annamie Paul era.\n\n(Makes a lot more sense than the other comments here, given the most likely second choice for the Greens is NDP, and second/third choice for NDP voters is the Greens).', 'I’m curious, and NOT trying to start a big thing, but why do people like Gore, Obama and various other climate change espousers buy beach front property that costs millions of dollars? Isn’t that shit going to be underwater?', "Not religious, and I live in the Atlanta area in the US. \n\nClimate change is going to cause a lot of harm to a lot of people, but it is not so dire that it's going to wipe out our futures. Especially here in the rich world. Unfortunately most of the harm is going to come to the world's poorest, those at risk of famine and more exposed to extreme heat. Mass migrations and fights over natural resources could get ugly. But they're not going to threaten civilization itself. \n\nAtlanta's weather is gonna look like present-day Jacksonville's if we get to worldwide 3C warming. Which is fine. I'm not planning on apocalyptic outcomes because the weather here looks like Jacksonville.\n\nAgain, we should do what we can to prevent climate change from happening. But if you think the world's gonna end, you're deluding yourself.", 'People like you make me root for unstoppable, extinction-level, climate change.', 'During this minute, people are being informed about climate change', "She's not a crazy person. She's super smart. She honestly probably does know more about the state budget than any other politician in Oregon.\n\nI'd never vote for her because her environmental record is pretty terrible. I don't know if she's a climate change denier per se, but she doesn't' really care about addressing it. But for some people that's probably a plus.", 'Climate change works on a sliding scale though. It is not black or white.', "It looks like OP posted an AMP link. These should load faster, but AMP is controversial because of [concerns over privacy and the Open Web](https://www.reddit.com/r/AmputatorBot/comments/ehrq3z/why_did_i_build_amputatorbot).\n\nMaybe check out **the canonical page** instead: **[https://www.cbc.ca/news/canada/nova-scotia/carbon-pricing-climate-change-greenhouse-gases-1.6566471](https://www.cbc.ca/news/canada/nova-scotia/carbon-pricing-climate-change-greenhouse-gases-1.6566471)**\n\n*****\n\n ^(I'm a bot | )[^(Why &amp; About)](https://www.reddit.com/r/AmputatorBot/comments/ehrq3z/why_did_i_build_amputatorbot)^( | )[^(Summon: u/AmputatorBot)](https://www.reddit.com/r/AmputatorBot/comments/cchly3/you_can_now_summon_amputatorbot/)", 'Spreading what bullshit? That what was once global warming was changed to climate change because global temps in many regions was cooling? And that climate change could be used regardless what “the weather” cycle was doing? It’s easy to look back at print and then digital docs that proved its all a money /power grab. They’ve been warming of a climate emergency for 60 years. According to Al Gore and other hoaxsters the state of Florida should be half underwater for the last 20 years, as should Manhattan lol', 'You guys aren’t getting how climate change works are you?', 'All I’m getting from you is “you can’t care about one topic unless you care about ALL topics”. Your posts are damn near impossible to read and maintain a coherent message. \n\nI do like how you blame capitalism and war for climate change. Especially since that’s not the case. It’s humanity that is to blame, not a mechanism of it.', 'I alread know of them but thanks. I had no idea they were about climate change, I just thought the members were vegan', 'This weather event does not imply the opposite of "warming" merely because that area of the world had a larger than average amount of ice buildup one day - you\'re not posting about "coverage of weather events", you\'re posting about a specific event and then making the claim that these events are ignored... they\'re not.. they\'re recorded and used to build a model of climate change, which indicates that we are warming the globe through human behaviors.. a model \\_you\\_ appear to be ignoring.', 'watching the earth suffer from the consequences of climate change knowing it could’ve been avoided if al gore won in 2000', "The pop went up a 100k in last 2 years. Most of that was inward migration. It doesn't matter a damn if the birth rate declines. With climate change in the mix, the pop could hit 10 million in 50 years. Those boxes that shit money are only going to get more valuable.", "The Earth's climate is always changing. It will change whether there are humans on it or not.", 'You know climate change is getting to a point where its irreversible right? (We might have even crossed the threshold)', '&gt;\tHowever her actual childhood hasn’t actually been affected by climate change\n\nhttps://www.politico.eu/article/excess-death-surged-heat-wave-hit-europe/amp/\n\nthisisfine.jpg', 'Yes, fuck you in general. Now please stop hanging around a reddit thread about thousands of people dying due to the apocalyptic effects of  unrestrained global warming and climate change. Find something better to di', 'Animals are precious, I’m a horticulture major who’s concerned about ecological deprivation and the rapid change in climate in the last 10 years alone. Also animal proteins are a bit harder on the digestive tract than their non-animal counterparts. Also a lot cheaper to eat vegan. Plus I have pretty bad contamination OCD and eating meat is a huge source of anxiety for me. Although that’s more of its own problem but I’m sure you get what I mean', '"While that may still be a concern, the larger concern today involves the overreach of the SCOTUS and the GOP\'s pivot towards more extreme positions relative the abortion debate (and the potential additional restrictions of freedoms could be taken by the SCOTUS and the GOP)."\n\nGiving the American people the right to vote on Abortion is not extreme.  What\'s extreme is taking away that vote by letting SCOTUS decide.\n\nAs a country we\'ve always believed that there are things not listed in the constitution and that states have a right and obligation to handle those things the way they see fit.  There\'s no right to Abortion in the U.S. constitution.\n\nThat being said, the only people concerned that Roe was overturned is the far left.  Right now Americans can\'t afford gas, food, and necessities (or those necessities are gone).  That\'s what they are focusing on and the left is doing nothing to fix it.  We are in a recession and all the left can do is spend money on "climate change" when they should be trying to get us out of this recession.', 'Climate change', "Nobody is going to listen to Kevin from the council estate, when he calls for climate change action.\n\nIf high-profile people can't call for action without living in a yurt, then they're not going to do it.\n\nBetter that they fly in private jets and call for action than they fly in private jets and pretend climate change isn't real (like right-wing politicians).", 'they should just halt it immediately, why wait? the climate isn’t waiting for us to change our ways', "&gt;Most of the deforestation is being done by large corporate   \nagribusinesses with international financing and that not only   \nirrevocably damages biodiversity (extinction is forever) and exacerbates  \n climate change\n\nHow is this different than what's going on in the US? And the regions of Brazil that are being deforested are also the regions of Brazil that voted for Bolsanaro.", 'Who is slaying dragons? Nobody. As of now, nothing is being done. We are headed straight to disaster. When you’re deciding whether to bring children into this world, look at reality, not what you think reality should be. I swear, people treat climate change like some kind of vaaaague idea that may or may not happen at some point.\n\nAlso, by bringing children into a world of increasingly scarce resources, you’re not only hurting the child but also becoming a cause of the problem.', "Yes, I think I have stated my question multiple times now: Most people don't currently support the changes needed to curb climate change due to the costs and trade offs involved. **Why would they suddenly support the changes needed if the economic system changed to socialism?** I've yet to hear an explanation of that anywhere here.", 'So many people here talking about climate change as the cause.. look closely. You can see at least 2 orcas moving the ice.', "So OP is talking about Consistent Trends versus Singular Events.  Like the difference between Climate Change and Weather.  It's literally watching people who cannot interpret how the definitions of words interact when placed in a sentence to communicate an event or idea...", 'Bernie considers climate crisis the most valuable change to national security. He opposed the manufacturing of the Dakota Access Pipeline on the grounds that it will have a notable result on the climate.\n___\nBernie Bot -Bernie_Bot_01', 'I\'ve argued EvC since the alt boards on monochromatic screens, and coming here I don\'t see any changes in their "arguments."\n\nOn top of that, anti-science is as strong as ever, from climate change deniers to anti-vaxers to essential oils, not to mention flat earthers who were back then a literal goof.\n\nAnd while religion may be in decline, here and there, rabid religious though is rising up. No one thought Roe would be over turned, not Texas is forcing schools to put up "in god we trust" signs, so I hardly expect "The push to get creationism in schools seems to have stopped" if it is the case, to be the case soon.\n\nIf you "ok boomer" this, you\'re in for the same disappointment every generation has.', "Don't forget how Toyota was lobbying against laws that would help with Climate Change.    \n\n\nSource: https://driving.ca/auto-news/industry/toyota-ranked-third-worst-company-for-lobbying-against-climate-policies-report", "Sure it wasn't climate change?", 'You know nothing of being wrecked. Relationships, or whatever you call them, pale in comparison to, fiancé’s father committing suicide. Then fiancé trying multiple times to do the dame or your own father dying of a heart attack in his sleep. Abuse from a narcissist ex boyfriend. Not to mention many other things much worse then that. \n\nThose expectations you had suck. No denying that. But its just on a whole different level then what other people deal with. Its like a first graders homework vs climate change.', 'This feels like that time Trump brought in a snowball to prove climate change wasnt real']

def test_4():
    """
    Test evaluation for sentiment between -2 and -1
    """
    reddit = Reddit('/content/drive/MyDrive/Projeto Guiado 2.1/the-reddit-climate-change-dataset-comments.csv')
    assert reddit.analized_sentiment(-2,-1) == None

def test_5():
    """
    Test evaluation for score 4
    """
    reddit = Reddit('/content/drive/MyDrive/Projeto Guiado 2.1/the-reddit-climate-change-dataset-comments.csv')
    assert reddit.score(4) == ['Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute'] 

def test_6():
    """
    Test evaluation for score 0
    """
    reddit = Reddit('/content/drive/MyDrive/Projeto Guiado 2.1/the-reddit-climate-change-dataset-comments.csv')
    assert reddit.score(0) == ['Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', "You are correct. I have no concern for climate change. I leave that in God's hands to not man's.\n\nHow would you express Christ to the world?"] 


Overwriting test_data.py


In [12]:
!pytest . -vv

Test session starts (platform: linux, Python 3.7.14, pytest 3.6.4, pytest-sugar 0.9.5)
cachedir: .pytest_cache
rootdir: /content, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_data.py::test_1 ✓                                           17% █▋        
 test_data.py::test_2 ✓                                           33% ███▍      
 test_data.py::test_3 ✓                                           50% █████     
 test_data.py::test_4 ✓                                           67% ██████▋   
 test_data.py::test_5 ✓                                           83% ████████▍ 
 test_data.py::test_6 ✓                                          100% ██████████

Results (808.01s):
       6 passed
